In [391]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.cluster import KMeans

import os
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import *
from sklearn. metrics import *
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.ensemble import RandomForestRegressor

# Reading and displaying our dataset for content based filtering

In [392]:
#read datasets
art_data = pd.read_csv('WikiArt-info.csv')

In [393]:
art_data.head()

,ID,Category,Artist,Title,Year,Image URL,Painting Info URL,Artist Info URL
0,58c6237dedc2c9c7dc0de1ae,Impressionism,Charles Courtney Curran,In the Luxembourg Garden,1889,https://uploads3.wikiart.org/00123/images/char...,https://www.wikiart.org/en/charles-courtney-cu...,https://www.wikiart.org/en/charles-courtney-cu...
1,577280dfedc2cb3880f28e76,Neo-Expressionism,Keith Haring,The Marriage of Heaven and Hell,1984,https://uploads1.wikiart.org/images/keith-hari...,https://www.wikiart.org/en/keith-haring/the-ma...,https://www.wikiart.org/en/keith-haring
2,57727f2dedc2cb3880ed5fa9,Post-Impressionism,Jozsef Rippl-Ronai,Uncle Piacsek in front of the Black Sideboard,1906,https://uploads3.wikiart.org/images/j-zsef-rip...,https://www.wikiart.org/en/jozsef-rippl-ronai/...,https://www.wikiart.org/en/jozsef-rippl-ronai
3,58d1240cedc2c94f900fc610,Cubism,Vadym Meller,Monk. For the Play &#39;Mazeppa&#39;,1920,https://uploads2.wikiart.org/00124/images/vady...,https://www.wikiart.org/en/vadym-meller/monk-f...,https://www.wikiart.org/en/vadym-meller
4,57727de7edc2cb3880e91f26,Romanticism,David Wilkie,The Defence of Sarago&#231;a,1828,https://uploads6.wikiart.org/images/david-wilk...,https://www.wikiart.org/en/david-wilkie/the-de...,https://www.wikiart.org/en/david-wilkie


In [394]:
art_anotation = pd.read_csv('WikiArt-Emotions-Ag4.csv')

In [395]:
art_anotation.head()

,ID,Style,Category,Artist,Title,Year,Is painting,Face/body,Ave. art rating,Art (image+title): agreeableness,...,TitleOnly: love,TitleOnly: optimism,TitleOnly: pessimism,TitleOnly: regret,TitleOnly: sadness,TitleOnly: shame,TitleOnly: shyness,TitleOnly: surprise,TitleOnly: trust,TitleOnly: neutral
0,58c6237dedc2c9c7dc0de1ae,Modern Art,Impressionism,Charles Courtney Curran,In the Luxembourg Garden,1889,yes,face,2.33,0,...,0,0,0,0,0,0,0,0,0,0
1,577280dfedc2cb3880f28e76,Modern Art,Neo-Expressionism,Keith Haring,The Marriage of Heaven and Hell,1984,yes,body,0.70,0,...,0,0,0,0,0,0,0,0,0,0
2,57727f2dedc2cb3880ed5fa9,Modern Art,Post-Impressionism,Jozsef Rippl-Ronai,Uncle Piacsek in front of the Black Sideboard,1906,yes,face,1.60,0,...,0,0,0,0,0,0,0,0,0,0
3,58d1240cedc2c94f900fc610,Modern Art,Cubism,Vadym Meller,Monk. For the Play &#39;Mazeppa&#39;,1920,yes,face,0.82,0,...,0,0,0,0,0,0,0,0,0,0
4,57727de7edc2cb3880e91f26,Post Renaissance Art,Romanticism,David Wilkie,The Defence of Sarago&#231;a,1828,yes,face,1.69,0,...,0,0,0,0,0,0,0,0,0,0


# Performing Exploratory Data Analysis

In [396]:
art_data_dropped = art_data.drop(columns=art_data.columns.difference(['ID','Image URL']))
#drop all colums in art_data except 'ID' and 'image URL' so as to merge image Url with  the 'art_annotation dataframe'

In [397]:
art_data_dropped.head()

,ID,Image URL
0,58c6237dedc2c9c7dc0de1ae,https://uploads3.wikiart.org/00123/images/char...
1,577280dfedc2cb3880f28e76,https://uploads1.wikiart.org/images/keith-hari...
2,57727f2dedc2cb3880ed5fa9,https://uploads3.wikiart.org/images/j-zsef-rip...
3,58d1240cedc2c94f900fc610,https://uploads2.wikiart.org/00124/images/vady...
4,57727de7edc2cb3880e91f26,https://uploads6.wikiart.org/images/david-wilk...


In [398]:
main_art = pd.merge(art_anotation, art_data_dropped, on='ID')

In [399]:
main_art.head()

,ID,Style,Category,Artist,Title,Year,Is painting,Face/body,Ave. art rating,Art (image+title): agreeableness,...,TitleOnly: optimism,TitleOnly: pessimism,TitleOnly: regret,TitleOnly: sadness,TitleOnly: shame,TitleOnly: shyness,TitleOnly: surprise,TitleOnly: trust,TitleOnly: neutral,Image URL
0,58c6237dedc2c9c7dc0de1ae,Modern Art,Impressionism,Charles Courtney Curran,In the Luxembourg Garden,1889,yes,face,2.33,0,...,0,0,0,0,0,0,0,0,0,https://uploads3.wikiart.org/00123/images/char...
1,577280dfedc2cb3880f28e76,Modern Art,Neo-Expressionism,Keith Haring,The Marriage of Heaven and Hell,1984,yes,body,0.70,0,...,0,0,0,0,0,0,0,0,0,https://uploads1.wikiart.org/images/keith-hari...
2,57727f2dedc2cb3880ed5fa9,Modern Art,Post-Impressionism,Jozsef Rippl-Ronai,Uncle Piacsek in front of the Black Sideboard,1906,yes,face,1.60,0,...,0,0,0,0,0,0,0,0,0,https://uploads3.wikiart.org/images/j-zsef-rip...
3,58d1240cedc2c94f900fc610,Modern Art,Cubism,Vadym Meller,Monk. For the Play &#39;Mazeppa&#39;,1920,yes,face,0.82,0,...,0,0,0,0,0,0,0,0,0,https://uploads2.wikiart.org/00124/images/vady...
4,57727de7edc2cb3880e91f26,Post Renaissance Art,Romanticism,David Wilkie,The Defence of Sarago&#231;a,1828,yes,face,1.69,0,...,0,0,0,0,0,0,0,0,0,https://uploads6.wikiart.org/images/david-wilk...


In [400]:
print(main_art.shape)
#This prints out the number of rows and columns in our dataset

(4105, 70)


In [401]:
print(main_art.columns)

Index(['ID', 'Style', 'Category', 'Artist', 'Title', 'Year', 'Is painting',
       'Face/body', 'Ave. art rating', 'Art (image+title): agreeableness',
       'Art (image+title): anger', 'Art (image+title): anticipation',
       'Art (image+title): arrogance', 'Art (image+title): disagreeableness',
       'Art (image+title): disgust', 'Art (image+title): fear',
       'Art (image+title): gratitude', 'Art (image+title): happiness',
       'Art (image+title): humility', 'Art (image+title): love',
       'Art (image+title): optimism', 'Art (image+title): pessimism',
       'Art (image+title): regret', 'Art (image+title): sadness',
       'Art (image+title): shame', 'Art (image+title): shyness',
       'Art (image+title): surprise', 'Art (image+title): trust',
       'Art (image+title): neutral', 'ImageOnly: agreeableness',
       'ImageOnly: anger', 'ImageOnly: anticipation', 'ImageOnly: arrogance',
       'ImageOnly: disagreeableness', 'ImageOnly: disgust', 'ImageOnly: fear',
       'Imag

In [402]:
#we need to drop the columns with annotations for title only and image only, we are only focusing on the annotations when both image and title are combined.
main_art = main_art.drop(columns=main_art.columns.difference(['ID', 'Style', 'Category', 'Artist', 'Title', 'Year', 'Is painting',
       'Face/body', 'Ave. art rating', 'Art (image+title): agreeableness',
       'Art (image+title): anger', 'Art (image+title): anticipation',
       'Art (image+title): arrogance', 'Art (image+title): disagreeableness',
       'Art (image+title): disgust', 'Art (image+title): fear',
       'Art (image+title): gratitude', 'Art (image+title): happiness',
       'Art (image+title): humility', 'Art (image+title): love',
       'Art (image+title): optimism', 'Art (image+title): pessimism',
       'Art (image+title): regret', 'Art (image+title): sadness',
       'Art (image+title): shame', 'Art (image+title): shyness',
       'Art (image+title): surprise', 'Art (image+title): trust',
       'Art (image+title): neutral','Image URL']))

In [403]:
main_art.head()

,ID,Style,Category,Artist,Title,Year,Is painting,Face/body,Ave. art rating,Art (image+title): agreeableness,...,Art (image+title): optimism,Art (image+title): pessimism,Art (image+title): regret,Art (image+title): sadness,Art (image+title): shame,Art (image+title): shyness,Art (image+title): surprise,Art (image+title): trust,Art (image+title): neutral,Image URL
0,58c6237dedc2c9c7dc0de1ae,Modern Art,Impressionism,Charles Courtney Curran,In the Luxembourg Garden,1889,yes,face,2.33,0,...,0,0,0,0,0,0,0,0,0,https://uploads3.wikiart.org/00123/images/char...
1,577280dfedc2cb3880f28e76,Modern Art,Neo-Expressionism,Keith Haring,The Marriage of Heaven and Hell,1984,yes,body,0.70,0,...,0,0,0,0,0,0,1,0,0,https://uploads1.wikiart.org/images/keith-hari...
2,57727f2dedc2cb3880ed5fa9,Modern Art,Post-Impressionism,Jozsef Rippl-Ronai,Uncle Piacsek in front of the Black Sideboard,1906,yes,face,1.60,0,...,0,0,0,0,0,0,0,0,0,https://uploads3.wikiart.org/images/j-zsef-rip...
3,58d1240cedc2c94f900fc610,Modern Art,Cubism,Vadym Meller,Monk. For the Play &#39;Mazeppa&#39;,1920,yes,face,0.82,0,...,0,0,0,0,0,0,0,0,0,https://uploads2.wikiart.org/00124/images/vady...
4,57727de7edc2cb3880e91f26,Post Renaissance Art,Romanticism,David Wilkie,The Defence of Sarago&#231;a,1828,yes,face,1.69,0,...,0,0,0,0,0,0,0,1,0,https://uploads6.wikiart.org/images/david-wilk...


In [404]:
print(main_art.shape)

(4105, 30)


In [405]:
print(main_art.columns)

Index(['ID', 'Style', 'Category', 'Artist', 'Title', 'Year', 'Is painting',
       'Face/body', 'Ave. art rating', 'Art (image+title): agreeableness',
       'Art (image+title): anger', 'Art (image+title): anticipation',
       'Art (image+title): arrogance', 'Art (image+title): disagreeableness',
       'Art (image+title): disgust', 'Art (image+title): fear',
       'Art (image+title): gratitude', 'Art (image+title): happiness',
       'Art (image+title): humility', 'Art (image+title): love',
       'Art (image+title): optimism', 'Art (image+title): pessimism',
       'Art (image+title): regret', 'Art (image+title): sadness',
       'Art (image+title): shame', 'Art (image+title): shyness',
       'Art (image+title): surprise', 'Art (image+title): trust',
       'Art (image+title): neutral', 'Image URL'],
      dtype='object')


In [406]:
#rename colums with 'Art (image+title with 'emotions')
main_art = main_art.rename(columns={'Art (image+title): agreeableness': 'Emotion:agreeableness',
       'Art (image+title): anger': 'Emotion:anger', 'Art (image+title): anticipation': 'Emotion:anticipation',
       'Art (image+title): arrogance': 'Emotion:arrogance', 'Art (image+title): disagreeableness': 'Emotion:disagreeableness',
       'Art (image+title): disgust': 'Emotion:disgust', 'Art (image+title): fear': 'Emotion:fear',
       'Art (image+title): gratitude': 'Emotion:gratitude', 'Art (image+title): happiness': 'Emotion:happiness',
       'Art (image+title): humility': 'Emotion:humility', 'Art (image+title): love': 'Emotion:love',
       'Art (image+title): optimism': 'Emotion:optimism', 'Art (image+title): pessimism': 'Emotion:pessimism',
       'Art (image+title): regret': 'Emotion:regret', 'Art (image+title): sadness': 'Emotion:sadness',
       'Art (image+title): shame': 'Emotion:shame', 'Art (image+title): shyness': 'Emotion:shyness',
       'Art (image+title): surprise': 'Emotion:surprise', 'Art (image+title): trust': 'Emotion:trust',
       'Art (image+title): neutral': 'Emotion:neutral'})

In [407]:
print(main_art.columns)

Index(['ID', 'Style', 'Category', 'Artist', 'Title', 'Year', 'Is painting',
       'Face/body', 'Ave. art rating', 'Emotion:agreeableness',
       'Emotion:anger', 'Emotion:anticipation', 'Emotion:arrogance',
       'Emotion:disagreeableness', 'Emotion:disgust', 'Emotion:fear',
       'Emotion:gratitude', 'Emotion:happiness', 'Emotion:humility',
       'Emotion:love', 'Emotion:optimism', 'Emotion:pessimism',
       'Emotion:regret', 'Emotion:sadness', 'Emotion:shame', 'Emotion:shyness',
       'Emotion:surprise', 'Emotion:trust', 'Emotion:neutral', 'Image URL'],
      dtype='object')


In [408]:
main_art.head()

,ID,Style,Category,Artist,Title,Year,Is painting,Face/body,Ave. art rating,Emotion:agreeableness,...,Emotion:optimism,Emotion:pessimism,Emotion:regret,Emotion:sadness,Emotion:shame,Emotion:shyness,Emotion:surprise,Emotion:trust,Emotion:neutral,Image URL
0,58c6237dedc2c9c7dc0de1ae,Modern Art,Impressionism,Charles Courtney Curran,In the Luxembourg Garden,1889,yes,face,2.33,0,...,0,0,0,0,0,0,0,0,0,https://uploads3.wikiart.org/00123/images/char...
1,577280dfedc2cb3880f28e76,Modern Art,Neo-Expressionism,Keith Haring,The Marriage of Heaven and Hell,1984,yes,body,0.70,0,...,0,0,0,0,0,0,1,0,0,https://uploads1.wikiart.org/images/keith-hari...
2,57727f2dedc2cb3880ed5fa9,Modern Art,Post-Impressionism,Jozsef Rippl-Ronai,Uncle Piacsek in front of the Black Sideboard,1906,yes,face,1.60,0,...,0,0,0,0,0,0,0,0,0,https://uploads3.wikiart.org/images/j-zsef-rip...
3,58d1240cedc2c94f900fc610,Modern Art,Cubism,Vadym Meller,Monk. For the Play &#39;Mazeppa&#39;,1920,yes,face,0.82,0,...,0,0,0,0,0,0,0,0,0,https://uploads2.wikiart.org/00124/images/vady...
4,57727de7edc2cb3880e91f26,Post Renaissance Art,Romanticism,David Wilkie,The Defence of Sarago&#231;a,1828,yes,face,1.69,0,...,0,0,0,0,0,0,0,1,0,https://uploads6.wikiart.org/images/david-wilk...


In [409]:
print(main_art.shape)

(4105, 30)


In [410]:
print(main_art.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4105 entries, 0 to 4104
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        4105 non-null   object 
 1   Style                     4105 non-null   object 
 2   Category                  4105 non-null   object 
 3   Artist                    4105 non-null   object 
 4   Title                     4105 non-null   object 
 5   Year                      4105 non-null   object 
 6   Is painting               4105 non-null   object 
 7   Face/body                 4105 non-null   object 
 8   Ave. art rating           4105 non-null   float64
 9   Emotion:agreeableness     4105 non-null   int64  
 10  Emotion:anger             4105 non-null   int64  
 11  Emotion:anticipation      4105 non-null   int64  
 12  Emotion:arrogance         4105 non-null   int64  
 13  Emotion:disagreeableness  4105 non-null   int64  
 14  Emotion:

In [411]:
main_art.describe()
#print some statistical features for our dataset

,Ave. art rating,Emotion:agreeableness,Emotion:anger,Emotion:anticipation,Emotion:arrogance,Emotion:disagreeableness,Emotion:disgust,Emotion:fear,Emotion:gratitude,Emotion:happiness,...,Emotion:love,Emotion:optimism,Emotion:pessimism,Emotion:regret,Emotion:sadness,Emotion:shame,Emotion:shyness,Emotion:surprise,Emotion:trust,Emotion:neutral
count,4105.000000,4105.000000,4105.000000,4105.000000,4105.000000,4105.000000,4105.000000,4105.000000,4105.000000,4105.00000,...,4105.000000,4105.000000,4105.000000,4105.000000,4105.000000,4105.000000,4105.000000,4105.000000,4105.000000,4105.000000
mean,0.884585,0.000974,0.010231,0.123021,0.014860,0.001705,0.013398,0.081851,0.010475,0.29525,...,0.064555,0.047259,0.008770,0.002436,0.073569,0.005847,0.000487,0.219001,0.171985,0.000244
std,0.850203,0.031204,0.100644,0.328501,0.121007,0.041264,0.114987,0.274172,0.101823,0.45621,...,0.245770,0.212219,0.093247,0.049302,0.261100,0.076248,0.022070,0.413620,0.377414,0.015608
min,-2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.270000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.960000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.600000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2.800000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [412]:
main_art.isnull().sum()
#check for missing values

ID                          0
Style                       0
Category                    0
Artist                      0
Title                       0
Year                        0
Is painting                 0
Face/body                   0
Ave. art rating             0
Emotion:agreeableness       0
Emotion:anger               0
Emotion:anticipation        0
Emotion:arrogance           0
Emotion:disagreeableness    0
Emotion:disgust             0
Emotion:fear                0
Emotion:gratitude           0
Emotion:happiness           0
Emotion:humility            0
Emotion:love                0
Emotion:optimism            0
Emotion:pessimism           0
Emotion:regret              0
Emotion:sadness             0
Emotion:shame               0
Emotion:shyness             0
Emotion:surprise            0
Emotion:trust               0
Emotion:neutral             0
Image URL                   0
dtype: int64

================================================================================

# Content based filtering model

Feature Selection

In [413]:
Cont = main_art[['ID', 'Style', 'Category', 'Title', 'Artist', 'Face/body']] #selecting key features for our content based filtering

In [414]:
Cont

,ID,Style,Category,Title,Artist,Face/body
0,58c6237dedc2c9c7dc0de1ae,Modern Art,Impressionism,In the Luxembourg Garden,Charles Courtney Curran,face
1,577280dfedc2cb3880f28e76,Modern Art,Neo-Expressionism,The Marriage of Heaven and Hell,Keith Haring,body
2,57727f2dedc2cb3880ed5fa9,Modern Art,Post-Impressionism,Uncle Piacsek in front of the Black Sideboard,Jozsef Rippl-Ronai,face
3,58d1240cedc2c94f900fc610,Modern Art,Cubism,Monk. For the Play &#39;Mazeppa&#39;,Vadym Meller,face
4,57727de7edc2cb3880e91f26,Post Renaissance Art,Romanticism,The Defence of Sarago&#231;a,David Wilkie,face
...,...,...,...,...,...,...
4100,577287aeedc2cb388007fba9,Modern Art,Color Field Painting,OE 260,Rupprecht Geiger,none
4101,57728001edc2cb3880efddcf,Modern Art,Surrealism,M&#225;quina de coser electro-sexual,Oscar Dominguez,body
4102,57728ac7edc2cb3880123cc2,Modern Art,Neo-Expressionism,Female Nude on a Kitchen Chair,Georg Baselitz,body
4103,57728412edc2cb3880fc9ff4,Modern Art,Expressionism,Apollinaire and His Friends,Marie Laurencin,face


In [415]:
Cont ['Tags'] = Cont['Style'] +', ' +Cont['Category'] + ', ' + Cont['Artist'] #merge style, category and artist into Tags

In [416]:
Cont

,ID,Style,Category,Title,Artist,Face/body,Tags
0,58c6237dedc2c9c7dc0de1ae,Modern Art,Impressionism,In the Luxembourg Garden,Charles Courtney Curran,face,"Modern Art, Impressionism, Charles Courtney Cu..."
1,577280dfedc2cb3880f28e76,Modern Art,Neo-Expressionism,The Marriage of Heaven and Hell,Keith Haring,body,"Modern Art, Neo-Expressionism, Keith Haring"
2,57727f2dedc2cb3880ed5fa9,Modern Art,Post-Impressionism,Uncle Piacsek in front of the Black Sideboard,Jozsef Rippl-Ronai,face,"Modern Art, Post-Impressionism, Jozsef Rippl-R..."
3,58d1240cedc2c94f900fc610,Modern Art,Cubism,Monk. For the Play &#39;Mazeppa&#39;,Vadym Meller,face,"Modern Art, Cubism, Vadym Meller"
4,57727de7edc2cb3880e91f26,Post Renaissance Art,Romanticism,The Defence of Sarago&#231;a,David Wilkie,face,"Post Renaissance Art, Romanticism, David Wilkie"
...,...,...,...,...,...,...,...
4100,577287aeedc2cb388007fba9,Modern Art,Color Field Painting,OE 260,Rupprecht Geiger,none,"Modern Art, Color Field Painting, Rupprecht Ge..."
4101,57728001edc2cb3880efddcf,Modern Art,Surrealism,M&#225;quina de coser electro-sexual,Oscar Dominguez,body,"Modern Art, Surrealism, Oscar Dominguez"
4102,57728ac7edc2cb3880123cc2,Modern Art,Neo-Expressionism,Female Nude on a Kitchen Chair,Georg Baselitz,body,"Modern Art, Neo-Expressionism, Georg Baselitz"
4103,57728412edc2cb3880fc9ff4,Modern Art,Expressionism,Apollinaire and His Friends,Marie Laurencin,face,"Modern Art, Expressionism, Marie Laurencin"


In [417]:
New_Cont= Cont.drop(columns=['Style','Category','Artist'])

In [418]:
New_Cont

,ID,Title,Face/body,Tags
0,58c6237dedc2c9c7dc0de1ae,In the Luxembourg Garden,face,"Modern Art, Impressionism, Charles Courtney Cu..."
1,577280dfedc2cb3880f28e76,The Marriage of Heaven and Hell,body,"Modern Art, Neo-Expressionism, Keith Haring"
2,57727f2dedc2cb3880ed5fa9,Uncle Piacsek in front of the Black Sideboard,face,"Modern Art, Post-Impressionism, Jozsef Rippl-R..."
3,58d1240cedc2c94f900fc610,Monk. For the Play &#39;Mazeppa&#39;,face,"Modern Art, Cubism, Vadym Meller"
4,57727de7edc2cb3880e91f26,The Defence of Sarago&#231;a,face,"Post Renaissance Art, Romanticism, David Wilkie"
...,...,...,...,...
4100,577287aeedc2cb388007fba9,OE 260,none,"Modern Art, Color Field Painting, Rupprecht Ge..."
4101,57728001edc2cb3880efddcf,M&#225;quina de coser electro-sexual,body,"Modern Art, Surrealism, Oscar Dominguez"
4102,57728ac7edc2cb3880123cc2,Female Nude on a Kitchen Chair,body,"Modern Art, Neo-Expressionism, Georg Baselitz"
4103,57728412edc2cb3880fc9ff4,Apollinaire and His Friends,face,"Modern Art, Expressionism, Marie Laurencin"


In [419]:
New_Cont.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4105 entries, 0 to 4104
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         4105 non-null   object
 1   Title      4105 non-null   object
 2   Face/body  4105 non-null   object
 3   Tags       4105 non-null   object
dtypes: object(4)
memory usage: 160.4+ KB


Clustering

In [420]:
# Apply TF-IDF vectorization to item features
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(New_Cont['Tags'])


In [421]:
print(tfidf_matrix)

  (0, 364)	0.5964392695133415
  (0, 350)	0.5964392695133415
  (0, 296)	0.45047162596395846
  (0, 719)	0.2574829929593943
  (0, 105)	0.07735148706579527
  (0, 1031)	0.11539049159940891
  (1, 653)	0.6604867729468491
  (1, 805)	0.6194492761818874
  (1, 484)	0.2347093440844306
  (1, 1077)	0.3227001550912016
  (1, 105)	0.08033562698444642
  (1, 1031)	0.11984213674905779
  (2, 1286)	0.5484121911970411
  (2, 1264)	0.5484121911970411
  (2, 777)	0.5484121911970411
  (2, 1212)	0.15926815025125296
  (2, 719)	0.23674969034156654
  (2, 105)	0.07112291340024432
  (2, 1031)	0.10609890323447271
  (3, 1000)	0.6582974268309618
  (3, 1507)	0.6582974268309618
  (3, 360)	0.33329274602798753
  (3, 105)	0.08297263357050141
  (3, 1031)	0.12377593941864239
  (4, 1576)	0.6898589178019229
  :	:
  (4100, 325)	0.3307815476494242
  (4100, 105)	0.08234747522136086
  (4100, 1031)	0.12284334804941065
  (4101, 419)	0.655545819567237
  (4101, 1123)	0.655545819567237
  (4101, 1432)	0.34235471253419
  (4101, 105)	0.085016

In [422]:
# Perform K-means clustering
num_clusters = 200
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
item_clusters = kmeans.fit_predict(tfidf_matrix)

In [423]:
# Calculate cosine similarity between items based on attributes
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [424]:

# Create a dictionary to map item IDs to cluster labels
item_to_cluster = dict(zip(New_Cont.index, item_clusters))

In [425]:
print(item_to_cluster)

{0: 89, 1: 22, 2: 29, 3: 115, 4: 27, 5: 13, 6: 15, 7: 89, 8: 66, 9: 137, 10: 53, 11: 174, 12: 76, 13: 76, 14: 115, 15: 131, 16: 123, 17: 54, 18: 109, 19: 16, 20: 140, 21: 19, 22: 0, 23: 173, 24: 14, 25: 22, 26: 137, 27: 20, 28: 21, 29: 137, 30: 76, 31: 73, 32: 82, 33: 138, 34: 70, 35: 89, 36: 144, 37: 40, 38: 33, 39: 55, 40: 75, 41: 75, 42: 29, 43: 148, 44: 143, 45: 64, 46: 22, 47: 67, 48: 39, 49: 75, 50: 197, 51: 153, 52: 22, 53: 119, 54: 65, 55: 67, 56: 26, 57: 167, 58: 44, 59: 34, 60: 53, 61: 29, 62: 29, 63: 20, 64: 93, 65: 39, 66: 42, 67: 133, 68: 10, 69: 167, 70: 13, 71: 29, 72: 89, 73: 106, 74: 106, 75: 14, 76: 5, 77: 133, 78: 4, 79: 52, 80: 129, 81: 21, 82: 40, 83: 75, 84: 16, 85: 185, 86: 39, 87: 44, 88: 178, 89: 21, 90: 185, 91: 154, 92: 67, 93: 98, 94: 118, 95: 53, 96: 160, 97: 22, 98: 160, 99: 4, 100: 29, 101: 36, 102: 76, 103: 83, 104: 154, 105: 8, 106: 137, 107: 22, 108: 81, 109: 126, 110: 29, 111: 173, 112: 47, 113: 39, 114: 14, 115: 29, 116: 20, 117: 137, 118: 19, 119: 1

In [426]:
New_Cont['clusters']=item_to_cluster

In [427]:
New_Cont.sample(15)

,ID,Title,Face/body,Tags,clusters
2282,57727015edc2cb3880bceb18,Still Life,none,"Post Renaissance Art, Baroque, Francisco de Zu...",73
2550,57726e75edc2cb3880b73c7b,Watson and the Shark,face,"Post Renaissance Art, Romanticism, John Single...",171
2472,577279a7edc2cb3880dbeac4,Painterly Architectonic,none,"Modern Art, Abstract Art, Lyubov Popova",75
2591,57727f93edc2cb3880ee399c,Safta the Flower Girl,face,"Modern Art, Post-Impressionism, Stefan Luchian",185
3347,57728170edc2cb3880f4683b,Etendue,none,"Modern Art, Lyrical Abstraction, Bram van Velde",118
468,57727da5edc2cb3880e868a8,Portrait of Mary Block,face,"Modern Art, Magic Realism, Ivan Albright",90
2715,57726e37edc2cb3880b627ee,Amor Victorious,face,"Post Renaissance Art, Baroque, Caravaggio",38
507,577275e5edc2cb3880cf8969,At the Edge of the Brook,face,"Post Renaissance Art, Neoclassicism, William-A...",39
2687,577288a2edc2cb38800b28b7,Unistic Composition,none,"Modern Art, Abstract Art, Wladyslaw Strzeminski",96
3458,59195e67edc2c946985973e0,The Rainbow,none,"Post Renaissance Art, Realism, Henry Moore",98


In [428]:

item_id = 390  # Replace with the item ID you want to check
cluster_id = item_to_cluster.get(item_id)

cluster_items = []

if cluster_id is not None:
    print(f"Item {item_id} belongs to Cluster {cluster_id}")

    # Find indices of items in the same cluster
    cluster_items_indices = np.where(item_clusters == cluster_id)[0]

    # Exclude the input item itself from the indices
    cluster_items_indices = cluster_items_indices[cluster_items_indices != item_id]

    # Get the item IDs of all items in the same cluster
    items_in_cluster = [item_id for item_id in item_to_cluster.keys() if item_to_cluster[item_id] == cluster_id]

    # Remove the input item from the items in the cluster
    items_in_cluster.remove(item_id)

    # Assign the item IDs to the cluster_items list
    cluster_items.extend(items_in_cluster)
else:
    print(f"Item {item_id} not found in any cluster.")

print('Items in this cluster are:')
# Print the saved cluster items (only item IDs) up to the top 8
for i, item in enumerate(cluster_items[:8], start=1):
    print(f"{i}. Item ID {item}")

# Assign the top 8 cluster item IDs to a variable
cluster_item_ids = cluster_items[:8]

print(cluster_item_ids)


Item 390 belongs to Cluster 63
Items in this cluster are:
1. Item ID 1011
2. Item ID 1068
3. Item ID 1163
4. Item ID 2746
5. Item ID 3285
6. Item ID 3333
7. Item ID 3505
8. Item ID 3558
[1011, 1068, 1163, 2746, 3285, 3333, 3505, 3558]


Content-based Algorithm

In [429]:
def recommend_with_cosine_similarity(item_id, num_recommendations=8):
    cluster_id = item_to_cluster.get(item_id)

    cluster_items = []

    if cluster_id is not None:
        print(f"Item {item_id} belongs to Cluster {cluster_id}")

        # Find indices of items in the same cluster
        cluster_items_indices = np.where(item_clusters == cluster_id)[0]

        # Exclude the input item itself from the indices
        cluster_items_indices = cluster_items_indices[cluster_items_indices != item_id]

        # Get the item IDs of all items in the same cluster
        items_in_cluster = [item_id for item_id in item_to_cluster.keys() if item_to_cluster[item_id] == cluster_id]
        # Remove the input item from the items in the cluster
        items_in_cluster.remove(item_id)

        # Assign the item IDs to the cluster_items list
        cluster_items.extend(items_in_cluster)
    else:
        print(f"Item {item_id} not found in any cluster.")

    print('Top 8 Items in this cluster are:')
    # Print the saved cluster items (only item IDs) up to the top 8
    for i, item in enumerate(cluster_items[:8], start=1):
        title = New_Cont['Title'].iloc[item]  # Replace 'Title' with the actual column name
        print(f"{i}. Item ID {item} - Title: {title}")  # Print title as well

    # Assign the top 8 cluster item IDs to a variable
    cluster_item_ids = cluster_items[:8]

    print(cluster_item_ids)

    # Calculate cosine similarity scores for items in the cluster
    sim_scores = list(enumerate(cosine_sim[item_id]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = [i for i in sim_scores if i[0] in cluster_items_indices]
    sim_scores = sim_scores[:num_recommendations]

    recommended_indices = [i[0] for i in sim_scores]

    print("\nRecommended Items:")
    for i, index in enumerate(recommended_indices):
        Id = main_art['ID'].iloc[index]
        title = New_Cont['Title'].iloc[index]  # Replace 'Title' with the actual column name
        image_url = main_art['Image URL'].iloc[index]
        style = main_art['Style'].iloc[index]
        category = main_art['Category'].iloc[index]
        artist = main_art['Artist'].iloc[index]
        #print(f"{i + 1}.Item ID {index} ID:{Id} - Title: {title} - Image URL: {image_url} - Style: {style} - Category: {category} - Artist: {artist}")  # Print title as well
        print(f"{i + 1}. Item ID {index}")
        print(f"   ID: {Id}")
        print(f"   Title: {title}")
        print(f"   Image URL: {image_url}")
        print(f"   Style: {style}")
        print(f"   Category: {category}")
        print(f"   Artist: {artist}")
    return cluster_item_ids, recommended_indices

# Choose a random item ID for testing
test_item_id = 390

# Get the recommendations using cosine similarity
cluster_item_ids, recommended_indices = recommend_with_cosine_similarity(test_item_id)

# Filter the items in the same cluster to only those in the top clusters
actual_items = cluster_item_ids
print("\nRECOMMENDED ITEMS INDEX:")
print(recommended_indices)
print("\nCLUSTER ITEMS INDEX:")
print(cluster_item_ids)
# Calculate precision and recall
precision = precision_score(actual_items, recommended_indices, average='micro')
recall = recall_score(actual_items, recommended_indices, average='micro')
f1 = f1_score(actual_items, recommended_indices, average='micro')

print(f"\nPrecision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")


Item 390 belongs to Cluster 63
Top 8 Items in this cluster are:
1. Item ID 1011 - Title: Encore une femme
2. Item ID 1068 - Title: I Was a Rich Man&#39;s Plaything
3. Item ID 1163 - Title: Dr. Pepper
4. Item ID 2746 - Title: La Petite
5. Item ID 3285 - Title: Meet the People
6. Item ID 3333 - Title: Velazquez, mon p&#232;re
7. Item ID 3505 - Title: Cyclops
8. Item ID 3558 - Title: Lou&#231;a de Barcelos
[1011, 1068, 1163, 2746, 3285, 3333, 3505, 3558]

Recommended Items:
1. Item ID 1011
   ID: 5772869aedc2cb388004cc10
   Title: Encore une femme
   Image URL: https://uploads6.wikiart.org/images/eduardo-arroyo/encore-une-femme-1970.jpg
   Style: Modern Art
   Category: Pop Art
   Artist: Eduardo Arroyo
2. Item ID 3333
   ID: 5772869bedc2cb388004ccf0
   Title: Velazquez, mon p&#232;re
   Image URL: https://uploads8.wikiart.org/images/eduardo-arroyo/velasquez-mon-p-re-1964.jpg
   Style: Modern Art
   Category: Pop Art
   Artist: Eduardo Arroyo
3. Item ID 3752
   ID: 5772869bedc2cb388004cd0

In [430]:
#import pickle

In [431]:
#pickle.dump(New_Cont, open('art_list.pkl','wb')) #creating the pickle file for our web app

In [432]:
#pickle.dump(New_Cont, open('similarity.pkl','wb'))

In [433]:
#pickle.load(open())

===============================================================================

# Reading and displaying our dataset for collaborative filtering

In [434]:
workers = pd.read_csv('WikiArt-annotations.csv')

In [435]:
workers.head()

,Worker ID,ImageOnly #1: ID,ImageOnly #1: Emotions,ImageOnly #1: Other Emotions,ImageOnly #2: ID,ImageOnly #2: Emotions,ImageOnly #2: Other Emotions,ImageOnly #3: ID,ImageOnly #3: Emotions,ImageOnly #3: Other Emotions,...,Art (image+title) #4: Other Emotions,Art (image+title) #4: Is painting,Art (image+title) #4: Face/body,Art (image+title) #4: Art rating,Art (image+title) #5: ID,Art (image+title) #5: Emotions,Art (image+title) #5: Other Emotions,Art (image+title) #5: Is painting,Art (image+title) #5: Face/body,Art (image+title) #5: Art rating
0,worker_1,57728030edc2cb3880f07e00,surprise,NaN,57726f0dedc2cb3880b94625,happiness\r\nlove,NaN,577284aeedc2cb3880fe8ca7,anticipation,NaN,...,NaN,yes,face\r\nbody,3,57728703edc2cb3880061039,trust,NaN,yes,none,3
1,worker_2,57728030edc2cb3880f07e00,anticipation,NaN,57726f0dedc2cb3880b94625,trust\r\nlove\r\narrogance\r\nanticipation,NaN,577284aeedc2cb3880fe8ca7,fear\r\nregret\r\nsadness\r\nshame,NaN,...,NaN,yes,face\r\nbody,3,57728703edc2cb3880061039,anger\r\nfear,NaN,yes,none,2
2,worker_3,57728030edc2cb3880f07e00,disgust,NaN,57726f0dedc2cb3880b94625,happiness,NaN,577284aeedc2cb3880fe8ca7,fear\r\ndisagreeableness,NaN,...,NaN,yes,body,1,57728703edc2cb3880061039,disgust,NaN,yes,none,0
3,worker_4,57728030edc2cb3880f07e00,anticipation,NaN,57726f0dedc2cb3880b94625,happiness\r\noptimism,NaN,577284aeedc2cb3880fe8ca7,sadness,NaN,...,NaN,yes,face\r\nbody,3,57728703edc2cb3880061039,surprise,NaN,yes,none,2
4,worker_5,57728030edc2cb3880f07e00,anticipation\r\nsurprise,NaN,57726f0dedc2cb3880b94625,happiness\r\nlove,NaN,577284aeedc2cb3880fe8ca7,anger\r\nsadness,NaN,...,NaN,yes,face\r\nbody,3,57728703edc2cb3880061039,love\r\nanger,NaN,yes,none,3


In [436]:
workers.shape

(12685, 61)

In [437]:
workers = workers.drop(columns=workers.columns.difference(['Worker ID','Art (image+title) #1: ID','Art (image+title) #1: Emotions','Art (image+title) #1: Face/body','Art (image+title) #1: Art rating','Art (image+title) #2: ID','Art (image+title) #2: Emotions','Art (image+title) #2: Face/body','Art (image+title) #2: Art rating','Art (image+title) #3: ID','Art (image+title) #3: Emotions','Art (image+title) #3: Face/body','Art (image+title) #3: Art rating','Art (image+title) #4: ID','Art (image+title) #4: Emotions','Art (image+title) #4: Face/body','Art (image+title) #4: Art rating','Art (image+title) #5: ID','Art (image+title) #5: Emotions','Art (image+title) #5: Face/body','Art (image+title) #5: Art rating',]))
#drop all art only and image only.

In [438]:
workers.head()

,Worker ID,Art (image+title) #1: ID,Art (image+title) #1: Emotions,Art (image+title) #1: Face/body,Art (image+title) #1: Art rating,Art (image+title) #2: ID,Art (image+title) #2: Emotions,Art (image+title) #2: Face/body,Art (image+title) #2: Art rating,Art (image+title) #3: ID,...,Art (image+title) #3: Face/body,Art (image+title) #3: Art rating,Art (image+title) #4: ID,Art (image+title) #4: Emotions,Art (image+title) #4: Face/body,Art (image+title) #4: Art rating,Art (image+title) #5: ID,Art (image+title) #5: Emotions,Art (image+title) #5: Face/body,Art (image+title) #5: Art rating
0,worker_1,577284aeedc2cb3880fe8ca7,anticipation,face\r\nbody,0,57728500edc2cb3880ffb2bc,disgust,none,-2,57728030edc2cb3880f07e00,...,none,0,57726f0dedc2cb3880b94625,happiness\r\nlove,face\r\nbody,3,57728703edc2cb3880061039,trust,none,3
1,worker_2,577284aeedc2cb3880fe8ca7,anger\r\ndisgust\r\nfear\r\nsadness,face\r\nbody,-2,57728500edc2cb3880ffb2bc,happiness,none,-3,57728030edc2cb3880f07e00,...,none,-3,57726f0dedc2cb3880b94625,trust\r\nhappiness\r\nlove\r\noptimism\r\narro...,face\r\nbody,3,57728703edc2cb3880061039,anger\r\nfear,none,2
2,worker_3,577284aeedc2cb3880fe8ca7,fear\r\ndisagreeableness,body,0,57728500edc2cb3880ffb2bc,disgust,none,0,57728030edc2cb3880f07e00,...,none,0,57726f0dedc2cb3880b94625,happiness\r\nlove,body,1,57728703edc2cb3880061039,disgust,none,0
3,worker_4,577284aeedc2cb3880fe8ca7,anger,face\r\nbody,3,57728500edc2cb3880ffb2bc,surprise,none,3,57728030edc2cb3880f07e00,...,none,3,57726f0dedc2cb3880b94625,happiness\r\noptimism,face\r\nbody,3,57728703edc2cb3880061039,surprise,none,2
4,worker_5,577284aeedc2cb3880fe8ca7,anger\r\nsadness,face\r\nbody,3,57728500edc2cb3880ffb2bc,fear,none,-1,57728030edc2cb3880f07e00,...,none,-2,57726f0dedc2cb3880b94625,happiness\r\nlove,face\r\nbody,3,57728703edc2cb3880061039,love\r\nanger,none,3


In [439]:
print(workers.columns)

Index(['Worker ID', 'Art (image+title) #1: ID',
       'Art (image+title) #1: Emotions', 'Art (image+title) #1: Face/body',
       'Art (image+title) #1: Art rating', 'Art (image+title) #2: ID',
       'Art (image+title) #2: Emotions', 'Art (image+title) #2: Face/body',
       'Art (image+title) #2: Art rating', 'Art (image+title) #3: ID',
       'Art (image+title) #3: Emotions', 'Art (image+title) #3: Face/body',
       'Art (image+title) #3: Art rating', 'Art (image+title) #4: ID',
       'Art (image+title) #4: Emotions', 'Art (image+title) #4: Face/body',
       'Art (image+title) #4: Art rating', 'Art (image+title) #5: ID',
       'Art (image+title) #5: Emotions', 'Art (image+title) #5: Face/body',
       'Art (image+title) #5: Art rating'],
      dtype='object')


In [440]:
df = pd.DataFrame(workers)

# Remove the common string from column names
common_string = '(image+title)'
df.columns = [col.replace(common_string, '') if i > 0 else col for i, col in enumerate(df.columns)]

# Display the modified DataFrame with updated column names
df.head()

,Worker ID,Art #1: ID,Art #1: Emotions,Art #1: Face/body,Art #1: Art rating,Art #2: ID,Art #2: Emotions,Art #2: Face/body,Art #2: Art rating,Art #3: ID,...,Art #3: Face/body,Art #3: Art rating,Art #4: ID,Art #4: Emotions,Art #4: Face/body,Art #4: Art rating,Art #5: ID,Art #5: Emotions,Art #5: Face/body,Art #5: Art rating
0,worker_1,577284aeedc2cb3880fe8ca7,anticipation,face\r\nbody,0,57728500edc2cb3880ffb2bc,disgust,none,-2,57728030edc2cb3880f07e00,...,none,0,57726f0dedc2cb3880b94625,happiness\r\nlove,face\r\nbody,3,57728703edc2cb3880061039,trust,none,3
1,worker_2,577284aeedc2cb3880fe8ca7,anger\r\ndisgust\r\nfear\r\nsadness,face\r\nbody,-2,57728500edc2cb3880ffb2bc,happiness,none,-3,57728030edc2cb3880f07e00,...,none,-3,57726f0dedc2cb3880b94625,trust\r\nhappiness\r\nlove\r\noptimism\r\narro...,face\r\nbody,3,57728703edc2cb3880061039,anger\r\nfear,none,2
2,worker_3,577284aeedc2cb3880fe8ca7,fear\r\ndisagreeableness,body,0,57728500edc2cb3880ffb2bc,disgust,none,0,57728030edc2cb3880f07e00,...,none,0,57726f0dedc2cb3880b94625,happiness\r\nlove,body,1,57728703edc2cb3880061039,disgust,none,0
3,worker_4,577284aeedc2cb3880fe8ca7,anger,face\r\nbody,3,57728500edc2cb3880ffb2bc,surprise,none,3,57728030edc2cb3880f07e00,...,none,3,57726f0dedc2cb3880b94625,happiness\r\noptimism,face\r\nbody,3,57728703edc2cb3880061039,surprise,none,2
4,worker_5,577284aeedc2cb3880fe8ca7,anger\r\nsadness,face\r\nbody,3,57728500edc2cb3880ffb2bc,fear,none,-1,57728030edc2cb3880f07e00,...,none,-2,57726f0dedc2cb3880b94625,happiness\r\nlove,face\r\nbody,3,57728703edc2cb3880061039,love\r\nanger,none,3


In [441]:

# Drop columns 'Face/body' since we would not be needing them
keyword = 'Face/body'
columns_to_drop = df.columns[df.columns.str.contains(keyword)]
df = df.drop(columns_to_drop, axis=1)
df.head()

,Worker ID,Art #1: ID,Art #1: Emotions,Art #1: Art rating,Art #2: ID,Art #2: Emotions,Art #2: Art rating,Art #3: ID,Art #3: Emotions,Art #3: Art rating,Art #4: ID,Art #4: Emotions,Art #4: Art rating,Art #5: ID,Art #5: Emotions,Art #5: Art rating
0,worker_1,577284aeedc2cb3880fe8ca7,anticipation,0,57728500edc2cb3880ffb2bc,disgust,-2,57728030edc2cb3880f07e00,surprise,0,57726f0dedc2cb3880b94625,happiness\r\nlove,3,57728703edc2cb3880061039,trust,3
1,worker_2,577284aeedc2cb3880fe8ca7,anger\r\ndisgust\r\nfear\r\nsadness,-2,57728500edc2cb3880ffb2bc,happiness,-3,57728030edc2cb3880f07e00,surprise,-3,57726f0dedc2cb3880b94625,trust\r\nhappiness\r\nlove\r\noptimism\r\narro...,3,57728703edc2cb3880061039,anger\r\nfear,2
2,worker_3,577284aeedc2cb3880fe8ca7,fear\r\ndisagreeableness,0,57728500edc2cb3880ffb2bc,disgust,0,57728030edc2cb3880f07e00,disgust,0,57726f0dedc2cb3880b94625,happiness\r\nlove,1,57728703edc2cb3880061039,disgust,0
3,worker_4,577284aeedc2cb3880fe8ca7,anger,3,57728500edc2cb3880ffb2bc,surprise,3,57728030edc2cb3880f07e00,surprise,3,57726f0dedc2cb3880b94625,happiness\r\noptimism,3,57728703edc2cb3880061039,surprise,2
4,worker_5,577284aeedc2cb3880fe8ca7,anger\r\nsadness,3,57728500edc2cb3880ffb2bc,fear,-1,57728030edc2cb3880f07e00,anticipation,-2,57726f0dedc2cb3880b94625,happiness\r\nlove,3,57728703edc2cb3880061039,love\r\nanger,3


In [442]:
df.shape


(12685, 16)

In [443]:
df['Worker ID'].value_counts() #count how many times each worker had to rate five pieces of art.

worker_6      305
worker_8      305
worker_3      298
worker_20     283
worker_33     270
             ... 
worker_238      5
worker_237      5
worker_236      5
worker_235      5
worker_307      5
Name: Worker ID, Length: 307, dtype: int64

In [444]:
print(df.columns)

Index(['Worker ID', 'Art  #1: ID', 'Art  #1: Emotions', 'Art  #1: Art rating',
       'Art  #2: ID', 'Art  #2: Emotions', 'Art  #2: Art rating',
       'Art  #3: ID', 'Art  #3: Emotions', 'Art  #3: Art rating',
       'Art  #4: ID', 'Art  #4: Emotions', 'Art  #4: Art rating',
       'Art  #5: ID', 'Art  #5: Emotions', 'Art  #5: Art rating'],
      dtype='object')


In [445]:
df = df.rename(columns={'Art  #1: Art rating': 'rating1'})
df = df.rename(columns={'Art  #2: Art rating': 'rating2'})
df = df.rename(columns={'Art  #3: Art rating': 'rating3'})
df = df.rename(columns={'Art  #4: Art rating': 'rating4'})
df = df.rename(columns={'Art  #5: Art rating': 'rating5'})

df = df.rename(columns={'Art  #1: ID': 'ID1'})
df = df.rename(columns={'Art  #2: ID': 'ID2'})
df = df.rename(columns={'Art  #3: ID': 'ID3'})
df = df.rename(columns={'Art  #4: ID': 'ID4'})
df = df.rename(columns={'Art  #5: ID': 'ID5'})

df = df.rename(columns={'Art  #1: Emotions': 'Emotions1'})
df = df.rename(columns={'Art  #2: Emotions': 'Emotions2'})
df = df.rename(columns={'Art  #3: Emotions': 'Emotions3'})
df = df.rename(columns={'Art  #4: Emotions': 'Emotions4'})
df = df.rename(columns={'Art  #5: Emotions': 'Emotions5'})

In [446]:
print(df.columns)

Index(['Worker ID', 'ID1', 'Emotions1', 'rating1', 'ID2', 'Emotions2',
       'rating2', 'ID3', 'Emotions3', 'rating3', 'ID4', 'Emotions4', 'rating4',
       'ID5', 'Emotions5', 'rating5'],
      dtype='object')


In [447]:
df.head()

,Worker ID,ID1,Emotions1,rating1,ID2,Emotions2,rating2,ID3,Emotions3,rating3,ID4,Emotions4,rating4,ID5,Emotions5,rating5
0,worker_1,577284aeedc2cb3880fe8ca7,anticipation,0,57728500edc2cb3880ffb2bc,disgust,-2,57728030edc2cb3880f07e00,surprise,0,57726f0dedc2cb3880b94625,happiness\r\nlove,3,57728703edc2cb3880061039,trust,3
1,worker_2,577284aeedc2cb3880fe8ca7,anger\r\ndisgust\r\nfear\r\nsadness,-2,57728500edc2cb3880ffb2bc,happiness,-3,57728030edc2cb3880f07e00,surprise,-3,57726f0dedc2cb3880b94625,trust\r\nhappiness\r\nlove\r\noptimism\r\narro...,3,57728703edc2cb3880061039,anger\r\nfear,2
2,worker_3,577284aeedc2cb3880fe8ca7,fear\r\ndisagreeableness,0,57728500edc2cb3880ffb2bc,disgust,0,57728030edc2cb3880f07e00,disgust,0,57726f0dedc2cb3880b94625,happiness\r\nlove,1,57728703edc2cb3880061039,disgust,0
3,worker_4,577284aeedc2cb3880fe8ca7,anger,3,57728500edc2cb3880ffb2bc,surprise,3,57728030edc2cb3880f07e00,surprise,3,57726f0dedc2cb3880b94625,happiness\r\noptimism,3,57728703edc2cb3880061039,surprise,2
4,worker_5,577284aeedc2cb3880fe8ca7,anger\r\nsadness,3,57728500edc2cb3880ffb2bc,fear,-1,57728030edc2cb3880f07e00,anticipation,-2,57726f0dedc2cb3880b94625,happiness\r\nlove,3,57728703edc2cb3880061039,love\r\nanger,3


In [448]:
df['Worker ID'].unique().shape #get how many workers gave rating

(307,)

In [449]:
x = df['Worker ID'].value_counts() >50 #select only workers with more than 50 ratings

In [450]:
x

worker_6       True
worker_8       True
worker_3       True
worker_20      True
worker_33      True
              ...  
worker_238    False
worker_237    False
worker_236    False
worker_235    False
worker_307    False
Name: Worker ID, Length: 307, dtype: bool

In [451]:
x[x].shape

(75,)

In [452]:
y = x[x].index
y

Index(['worker_6', 'worker_8', 'worker_3', 'worker_20', 'worker_33',
       'worker_24', 'worker_43', 'worker_83', 'worker_7', 'worker_40',
       'worker_15', 'worker_11', 'worker_46', 'worker_58', 'worker_37',
       'worker_26', 'worker_67', 'worker_4', 'worker_9', 'worker_28',
       'worker_19', 'worker_30', 'worker_38', 'worker_36', 'worker_56',
       'worker_51', 'worker_14', 'worker_50', 'worker_22', 'worker_99',
       'worker_49', 'worker_59', 'worker_27', 'worker_17', 'worker_29',
       'worker_25', 'worker_91', 'worker_5', 'worker_45', 'worker_10',
       'worker_87', 'worker_114', 'worker_104', 'worker_117', 'worker_77',
       'worker_16', 'worker_57', 'worker_90', 'worker_103', 'worker_18',
       'worker_65', 'worker_1', 'worker_23', 'worker_21', 'worker_131',
       'worker_48', 'worker_44', 'worker_13', 'worker_101', 'worker_63',
       'worker_84', 'worker_79', 'worker_111', 'worker_69', 'worker_149',
       'worker_107', 'worker_31', 'worker_92', 'worker_153', 'wo

In [453]:
df = df[df['Worker ID'].isin(y)]
df.head()

,Worker ID,ID1,Emotions1,rating1,ID2,Emotions2,rating2,ID3,Emotions3,rating3,ID4,Emotions4,rating4,ID5,Emotions5,rating5
0,worker_1,577284aeedc2cb3880fe8ca7,anticipation,0,57728500edc2cb3880ffb2bc,disgust,-2,57728030edc2cb3880f07e00,surprise,0,57726f0dedc2cb3880b94625,happiness\r\nlove,3,57728703edc2cb3880061039,trust,3
2,worker_3,577284aeedc2cb3880fe8ca7,fear\r\ndisagreeableness,0,57728500edc2cb3880ffb2bc,disgust,0,57728030edc2cb3880f07e00,disgust,0,57726f0dedc2cb3880b94625,happiness\r\nlove,1,57728703edc2cb3880061039,disgust,0
3,worker_4,577284aeedc2cb3880fe8ca7,anger,3,57728500edc2cb3880ffb2bc,surprise,3,57728030edc2cb3880f07e00,surprise,3,57726f0dedc2cb3880b94625,happiness\r\noptimism,3,57728703edc2cb3880061039,surprise,2
4,worker_5,577284aeedc2cb3880fe8ca7,anger\r\nsadness,3,57728500edc2cb3880ffb2bc,fear,-1,57728030edc2cb3880f07e00,anticipation,-2,57726f0dedc2cb3880b94625,happiness\r\nlove,3,57728703edc2cb3880061039,love\r\nanger,3
5,worker_6,577284aeedc2cb3880fe8ca7,trust,1,57728500edc2cb3880ffb2bc,disgust,-1,57728030edc2cb3880f07e00,disgust,-1,57726f0dedc2cb3880b94625,trust,1,57728703edc2cb3880061039,disgust,-2


In [454]:
df.shape

(9279, 16)

In [455]:
New_Cont.head() #Remember our data frame for collaborative filtering?

,ID,Title,Face/body,Tags,clusters
0,58c6237dedc2c9c7dc0de1ae,In the Luxembourg Garden,face,"Modern Art, Impressionism, Charles Courtney Cu...",89
1,577280dfedc2cb3880f28e76,The Marriage of Heaven and Hell,body,"Modern Art, Neo-Expressionism, Keith Haring",22
2,57727f2dedc2cb3880ed5fa9,Uncle Piacsek in front of the Black Sideboard,face,"Modern Art, Post-Impressionism, Jozsef Rippl-R...",29
3,58d1240cedc2c94f900fc610,Monk. For the Play &#39;Mazeppa&#39;,face,"Modern Art, Cubism, Vadym Meller",115
4,57727de7edc2cb3880e91f26,The Defence of Sarago&#231;a,face,"Post Renaissance Art, Romanticism, David Wilkie",27


In [456]:
#we want to merge it with our df' data frame so we can get the title for each ID
merged_df = df
for i in range(1, 6):
    column_name = f'ID{i}'
    merged_df = pd.merge(merged_df, New_Cont[['ID', 'Title','Tags']], left_on=column_name, right_on='ID', how='left', suffixes=('', f'_ID{i}'))
    merged_df = merged_df.drop(columns=['ID'])

merged_df.head()

,Worker ID,ID1,Emotions1,rating1,ID2,Emotions2,rating2,ID3,Emotions3,rating3,...,Title,Tags,Title_ID2,Tags_ID2,Title_ID3,Tags_ID3,Title_ID4,Tags_ID4,Title_ID5,Tags_ID5
0,worker_1,577284aeedc2cb3880fe8ca7,anticipation,0,57728500edc2cb3880ffb2bc,disgust,-2,57728030edc2cb3880f07e00,surprise,0,...,The Fight between the Lapiths and the Centaurs,"Renaissance Art, High Renaissance, Piero di Co...",Rhythmic Composition,"Modern Art, Art Informel, Ernst Wilhelm Nay",A Wall for Apricots,"Contemporary Art, Minimalism, Anne Truitt",The Love Letter,"Post Renaissance Art, Rococo, Jean-Honore Frag...",Sacred Flame,"Modern Art, Lyrical Abstraction, Kazuo Shiraga"
1,worker_3,577284aeedc2cb3880fe8ca7,fear\r\ndisagreeableness,0,57728500edc2cb3880ffb2bc,disgust,0,57728030edc2cb3880f07e00,disgust,0,...,The Fight between the Lapiths and the Centaurs,"Renaissance Art, High Renaissance, Piero di Co...",Rhythmic Composition,"Modern Art, Art Informel, Ernst Wilhelm Nay",A Wall for Apricots,"Contemporary Art, Minimalism, Anne Truitt",The Love Letter,"Post Renaissance Art, Rococo, Jean-Honore Frag...",Sacred Flame,"Modern Art, Lyrical Abstraction, Kazuo Shiraga"
2,worker_4,577284aeedc2cb3880fe8ca7,anger,3,57728500edc2cb3880ffb2bc,surprise,3,57728030edc2cb3880f07e00,surprise,3,...,The Fight between the Lapiths and the Centaurs,"Renaissance Art, High Renaissance, Piero di Co...",Rhythmic Composition,"Modern Art, Art Informel, Ernst Wilhelm Nay",A Wall for Apricots,"Contemporary Art, Minimalism, Anne Truitt",The Love Letter,"Post Renaissance Art, Rococo, Jean-Honore Frag...",Sacred Flame,"Modern Art, Lyrical Abstraction, Kazuo Shiraga"
3,worker_5,577284aeedc2cb3880fe8ca7,anger\r\nsadness,3,57728500edc2cb3880ffb2bc,fear,-1,57728030edc2cb3880f07e00,anticipation,-2,...,The Fight between the Lapiths and the Centaurs,"Renaissance Art, High Renaissance, Piero di Co...",Rhythmic Composition,"Modern Art, Art Informel, Ernst Wilhelm Nay",A Wall for Apricots,"Contemporary Art, Minimalism, Anne Truitt",The Love Letter,"Post Renaissance Art, Rococo, Jean-Honore Frag...",Sacred Flame,"Modern Art, Lyrical Abstraction, Kazuo Shiraga"
4,worker_6,577284aeedc2cb3880fe8ca7,trust,1,57728500edc2cb3880ffb2bc,disgust,-1,57728030edc2cb3880f07e00,disgust,-1,...,The Fight between the Lapiths and the Centaurs,"Renaissance Art, High Renaissance, Piero di Co...",Rhythmic Composition,"Modern Art, Art Informel, Ernst Wilhelm Nay",A Wall for Apricots,"Contemporary Art, Minimalism, Anne Truitt",The Love Letter,"Post Renaissance Art, Rococo, Jean-Honore Frag...",Sacred Flame,"Modern Art, Lyrical Abstraction, Kazuo Shiraga"


In [457]:
merged_df.columns

Index(['Worker ID', 'ID1', 'Emotions1', 'rating1', 'ID2', 'Emotions2',
       'rating2', 'ID3', 'Emotions3', 'rating3', 'ID4', 'Emotions4', 'rating4',
       'ID5', 'Emotions5', 'rating5', 'Title', 'Tags', 'Title_ID2', 'Tags_ID2',
       'Title_ID3', 'Tags_ID3', 'Title_ID4', 'Tags_ID4', 'Title_ID5',
       'Tags_ID5'],
      dtype='object')

In [458]:
#Rename 'Title' and'Tags' for ID1 for match the rest
merged_df = merged_df.rename({'Title': 'Title_ID1', 'Tags': 'Tags_ID1'}, axis=1)

In [459]:
merged_df.head()

,Worker ID,ID1,Emotions1,rating1,ID2,Emotions2,rating2,ID3,Emotions3,rating3,...,Title_ID1,Tags_ID1,Title_ID2,Tags_ID2,Title_ID3,Tags_ID3,Title_ID4,Tags_ID4,Title_ID5,Tags_ID5
0,worker_1,577284aeedc2cb3880fe8ca7,anticipation,0,57728500edc2cb3880ffb2bc,disgust,-2,57728030edc2cb3880f07e00,surprise,0,...,The Fight between the Lapiths and the Centaurs,"Renaissance Art, High Renaissance, Piero di Co...",Rhythmic Composition,"Modern Art, Art Informel, Ernst Wilhelm Nay",A Wall for Apricots,"Contemporary Art, Minimalism, Anne Truitt",The Love Letter,"Post Renaissance Art, Rococo, Jean-Honore Frag...",Sacred Flame,"Modern Art, Lyrical Abstraction, Kazuo Shiraga"
1,worker_3,577284aeedc2cb3880fe8ca7,fear\r\ndisagreeableness,0,57728500edc2cb3880ffb2bc,disgust,0,57728030edc2cb3880f07e00,disgust,0,...,The Fight between the Lapiths and the Centaurs,"Renaissance Art, High Renaissance, Piero di Co...",Rhythmic Composition,"Modern Art, Art Informel, Ernst Wilhelm Nay",A Wall for Apricots,"Contemporary Art, Minimalism, Anne Truitt",The Love Letter,"Post Renaissance Art, Rococo, Jean-Honore Frag...",Sacred Flame,"Modern Art, Lyrical Abstraction, Kazuo Shiraga"
2,worker_4,577284aeedc2cb3880fe8ca7,anger,3,57728500edc2cb3880ffb2bc,surprise,3,57728030edc2cb3880f07e00,surprise,3,...,The Fight between the Lapiths and the Centaurs,"Renaissance Art, High Renaissance, Piero di Co...",Rhythmic Composition,"Modern Art, Art Informel, Ernst Wilhelm Nay",A Wall for Apricots,"Contemporary Art, Minimalism, Anne Truitt",The Love Letter,"Post Renaissance Art, Rococo, Jean-Honore Frag...",Sacred Flame,"Modern Art, Lyrical Abstraction, Kazuo Shiraga"
3,worker_5,577284aeedc2cb3880fe8ca7,anger\r\nsadness,3,57728500edc2cb3880ffb2bc,fear,-1,57728030edc2cb3880f07e00,anticipation,-2,...,The Fight between the Lapiths and the Centaurs,"Renaissance Art, High Renaissance, Piero di Co...",Rhythmic Composition,"Modern Art, Art Informel, Ernst Wilhelm Nay",A Wall for Apricots,"Contemporary Art, Minimalism, Anne Truitt",The Love Letter,"Post Renaissance Art, Rococo, Jean-Honore Frag...",Sacred Flame,"Modern Art, Lyrical Abstraction, Kazuo Shiraga"
4,worker_6,577284aeedc2cb3880fe8ca7,trust,1,57728500edc2cb3880ffb2bc,disgust,-1,57728030edc2cb3880f07e00,disgust,-1,...,The Fight between the Lapiths and the Centaurs,"Renaissance Art, High Renaissance, Piero di Co...",Rhythmic Composition,"Modern Art, Art Informel, Ernst Wilhelm Nay",A Wall for Apricots,"Contemporary Art, Minimalism, Anne Truitt",The Love Letter,"Post Renaissance Art, Rococo, Jean-Honore Frag...",Sacred Flame,"Modern Art, Lyrical Abstraction, Kazuo Shiraga"


In [460]:
#Rearange our dataframe accordingly
column_order=['Worker ID', 'ID1', 'Title_ID1', 'Tags_ID1', 'Emotions1', 'rating1', 'ID2', 'Title_ID2', 'Tags_ID2', 'Emotions2', 'rating2', 'ID3', 'Title_ID3', 'Tags_ID3', 'Emotions3', 'rating3', 'ID4', 'Title_ID4', 'Tags_ID4', 'Emotions4', 'rating4', 'ID5', 'Title_ID5', 'Tags_ID5', 'Emotions5', 'rating5' ]
merged_df = merged_df[column_order]
merged_df.head()

,Worker ID,ID1,Title_ID1,Tags_ID1,Emotions1,rating1,ID2,Title_ID2,Tags_ID2,Emotions2,...,ID4,Title_ID4,Tags_ID4,Emotions4,rating4,ID5,Title_ID5,Tags_ID5,Emotions5,rating5
0,worker_1,577284aeedc2cb3880fe8ca7,The Fight between the Lapiths and the Centaurs,"Renaissance Art, High Renaissance, Piero di Co...",anticipation,0,57728500edc2cb3880ffb2bc,Rhythmic Composition,"Modern Art, Art Informel, Ernst Wilhelm Nay",disgust,...,57726f0dedc2cb3880b94625,The Love Letter,"Post Renaissance Art, Rococo, Jean-Honore Frag...",happiness\r\nlove,3,57728703edc2cb3880061039,Sacred Flame,"Modern Art, Lyrical Abstraction, Kazuo Shiraga",trust,3
1,worker_3,577284aeedc2cb3880fe8ca7,The Fight between the Lapiths and the Centaurs,"Renaissance Art, High Renaissance, Piero di Co...",fear\r\ndisagreeableness,0,57728500edc2cb3880ffb2bc,Rhythmic Composition,"Modern Art, Art Informel, Ernst Wilhelm Nay",disgust,...,57726f0dedc2cb3880b94625,The Love Letter,"Post Renaissance Art, Rococo, Jean-Honore Frag...",happiness\r\nlove,1,57728703edc2cb3880061039,Sacred Flame,"Modern Art, Lyrical Abstraction, Kazuo Shiraga",disgust,0
2,worker_4,577284aeedc2cb3880fe8ca7,The Fight between the Lapiths and the Centaurs,"Renaissance Art, High Renaissance, Piero di Co...",anger,3,57728500edc2cb3880ffb2bc,Rhythmic Composition,"Modern Art, Art Informel, Ernst Wilhelm Nay",surprise,...,57726f0dedc2cb3880b94625,The Love Letter,"Post Renaissance Art, Rococo, Jean-Honore Frag...",happiness\r\noptimism,3,57728703edc2cb3880061039,Sacred Flame,"Modern Art, Lyrical Abstraction, Kazuo Shiraga",surprise,2
3,worker_5,577284aeedc2cb3880fe8ca7,The Fight between the Lapiths and the Centaurs,"Renaissance Art, High Renaissance, Piero di Co...",anger\r\nsadness,3,57728500edc2cb3880ffb2bc,Rhythmic Composition,"Modern Art, Art Informel, Ernst Wilhelm Nay",fear,...,57726f0dedc2cb3880b94625,The Love Letter,"Post Renaissance Art, Rococo, Jean-Honore Frag...",happiness\r\nlove,3,57728703edc2cb3880061039,Sacred Flame,"Modern Art, Lyrical Abstraction, Kazuo Shiraga",love\r\nanger,3
4,worker_6,577284aeedc2cb3880fe8ca7,The Fight between the Lapiths and the Centaurs,"Renaissance Art, High Renaissance, Piero di Co...",trust,1,57728500edc2cb3880ffb2bc,Rhythmic Composition,"Modern Art, Art Informel, Ernst Wilhelm Nay",disgust,...,57726f0dedc2cb3880b94625,The Love Letter,"Post Renaissance Art, Rococo, Jean-Honore Frag...",trust,1,57728703edc2cb3880061039,Sacred Flame,"Modern Art, Lyrical Abstraction, Kazuo Shiraga",disgust,-2


In [461]:
Temp_merged = merged_df[['ID1','Title_ID1','ID2','Title_ID2','ID3','Title_ID3','ID4','Title_ID4','ID5','Title_ID5']]
Temp_merged.head()

,ID1,Title_ID1,ID2,Title_ID2,ID3,Title_ID3,ID4,Title_ID4,ID5,Title_ID5
0,577284aeedc2cb3880fe8ca7,The Fight between the Lapiths and the Centaurs,57728500edc2cb3880ffb2bc,Rhythmic Composition,57728030edc2cb3880f07e00,A Wall for Apricots,57726f0dedc2cb3880b94625,The Love Letter,57728703edc2cb3880061039,Sacred Flame
1,577284aeedc2cb3880fe8ca7,The Fight between the Lapiths and the Centaurs,57728500edc2cb3880ffb2bc,Rhythmic Composition,57728030edc2cb3880f07e00,A Wall for Apricots,57726f0dedc2cb3880b94625,The Love Letter,57728703edc2cb3880061039,Sacred Flame
2,577284aeedc2cb3880fe8ca7,The Fight between the Lapiths and the Centaurs,57728500edc2cb3880ffb2bc,Rhythmic Composition,57728030edc2cb3880f07e00,A Wall for Apricots,57726f0dedc2cb3880b94625,The Love Letter,57728703edc2cb3880061039,Sacred Flame
3,577284aeedc2cb3880fe8ca7,The Fight between the Lapiths and the Centaurs,57728500edc2cb3880ffb2bc,Rhythmic Composition,57728030edc2cb3880f07e00,A Wall for Apricots,57726f0dedc2cb3880b94625,The Love Letter,57728703edc2cb3880061039,Sacred Flame
4,577284aeedc2cb3880fe8ca7,The Fight between the Lapiths and the Centaurs,57728500edc2cb3880ffb2bc,Rhythmic Composition,57728030edc2cb3880f07e00,A Wall for Apricots,57726f0dedc2cb3880b94625,The Love Letter,57728703edc2cb3880061039,Sacred Flame


In [462]:
merged_df.shape

(9279, 26)

In [463]:
Art_rating_num = merged_df.groupby(['Title_ID1', 'Title_ID2', 'Title_ID3', 'Title_ID4', 'Title_ID5'])[['rating1', 'rating2', 'rating3', 'rating4', 'rating5']].count().reset_index()


In [464]:
Art_rating_num.head()

,Title_ID1,Title_ID2,Title_ID3,Title_ID4,Title_ID5,rating1,rating2,rating3,rating4,rating5
0,#14,Small table (upper surface),Fold,Landscape with the Flight,Bomen,9,9,9,9,9
1,15 octobre,The Skating Minister (The Reverend Robert Walk...,Agrigente,Untitled,The Sistine Madonna,9,9,9,9,9
2,15.Febr.62,Blanche rose,Family Reunion,Bronze Ballet,Composition VI,9,9,9,9,9
3,17. 3. 69,Primordial Fire,The Angler,Contemplation of a drawing,Che Guevara,10,10,10,10,10
4,18-1963,The Apotheosis of Homer,Triptyque ocre-violet-rouge,Levez le doigt et tout l&#39;univers est l&#224;,Landscape with river near Dordrecht,11,11,11,11,11


In [465]:
Art_rating_num.rename(columns={"rating1":"num_of_rating1", "rating2":"num_of_rating2", "rating3":"num_of_rating3", "rating4":"num_of_rating4", "rating5":"num_of_rating5"}, inplace=True)

In [466]:
Art_rating_num.head(2)

,Title_ID1,Title_ID2,Title_ID3,Title_ID4,Title_ID5,num_of_rating1,num_of_rating2,num_of_rating3,num_of_rating4,num_of_rating5
0,#14,Small table (upper surface),Fold,Landscape with the Flight,Bomen,9,9,9,9,9
1,15 octobre,The Skating Minister (The Reverend Robert Walk...,Agrigente,Untitled,The Sistine Madonna,9,9,9,9,9


In [467]:
merged_df.head(1)

,Worker ID,ID1,Title_ID1,Tags_ID1,Emotions1,rating1,ID2,Title_ID2,Tags_ID2,Emotions2,...,ID4,Title_ID4,Tags_ID4,Emotions4,rating4,ID5,Title_ID5,Tags_ID5,Emotions5,rating5
0,worker_1,577284aeedc2cb3880fe8ca7,The Fight between the Lapiths and the Centaurs,"Renaissance Art, High Renaissance, Piero di Co...",anticipation,0,57728500edc2cb3880ffb2bc,Rhythmic Composition,"Modern Art, Art Informel, Ernst Wilhelm Nay",disgust,...,57726f0dedc2cb3880b94625,The Love Letter,"Post Renaissance Art, Rococo, Jean-Honore Frag...",happiness\r\nlove,3,57728703edc2cb3880061039,Sacred Flame,"Modern Art, Lyrical Abstraction, Kazuo Shiraga",trust,3


In [468]:
Art_rating_num = merged_df.merge(Art_rating_num, on=['Title_ID1', 'Title_ID2', 'Title_ID3', 'Title_ID4', 'Title_ID5'])
Art_rating_num.head()

,Worker ID,ID1,Title_ID1,Tags_ID1,Emotions1,rating1,ID2,Title_ID2,Tags_ID2,Emotions2,...,ID5,Title_ID5,Tags_ID5,Emotions5,rating5,num_of_rating1,num_of_rating2,num_of_rating3,num_of_rating4,num_of_rating5
0,worker_1,577284aeedc2cb3880fe8ca7,The Fight between the Lapiths and the Centaurs,"Renaissance Art, High Renaissance, Piero di Co...",anticipation,0,57728500edc2cb3880ffb2bc,Rhythmic Composition,"Modern Art, Art Informel, Ernst Wilhelm Nay",disgust,...,57728703edc2cb3880061039,Sacred Flame,"Modern Art, Lyrical Abstraction, Kazuo Shiraga",trust,3,9,9,9,9,9
1,worker_3,577284aeedc2cb3880fe8ca7,The Fight between the Lapiths and the Centaurs,"Renaissance Art, High Renaissance, Piero di Co...",fear\r\ndisagreeableness,0,57728500edc2cb3880ffb2bc,Rhythmic Composition,"Modern Art, Art Informel, Ernst Wilhelm Nay",disgust,...,57728703edc2cb3880061039,Sacred Flame,"Modern Art, Lyrical Abstraction, Kazuo Shiraga",disgust,0,9,9,9,9,9
2,worker_4,577284aeedc2cb3880fe8ca7,The Fight between the Lapiths and the Centaurs,"Renaissance Art, High Renaissance, Piero di Co...",anger,3,57728500edc2cb3880ffb2bc,Rhythmic Composition,"Modern Art, Art Informel, Ernst Wilhelm Nay",surprise,...,57728703edc2cb3880061039,Sacred Flame,"Modern Art, Lyrical Abstraction, Kazuo Shiraga",surprise,2,9,9,9,9,9
3,worker_5,577284aeedc2cb3880fe8ca7,The Fight between the Lapiths and the Centaurs,"Renaissance Art, High Renaissance, Piero di Co...",anger\r\nsadness,3,57728500edc2cb3880ffb2bc,Rhythmic Composition,"Modern Art, Art Informel, Ernst Wilhelm Nay",fear,...,57728703edc2cb3880061039,Sacred Flame,"Modern Art, Lyrical Abstraction, Kazuo Shiraga",love\r\nanger,3,9,9,9,9,9
4,worker_6,577284aeedc2cb3880fe8ca7,The Fight between the Lapiths and the Centaurs,"Renaissance Art, High Renaissance, Piero di Co...",trust,1,57728500edc2cb3880ffb2bc,Rhythmic Composition,"Modern Art, Art Informel, Ernst Wilhelm Nay",disgust,...,57728703edc2cb3880061039,Sacred Flame,"Modern Art, Lyrical Abstraction, Kazuo Shiraga",disgust,-2,9,9,9,9,9


In [469]:
#Merge all titles and ratings, then calculate the number of rating counts

# Create an empty column 'Title'
Art_rating_num['Title'] = ""
Art_rating_num['rating_Count'] = ""

# Stack the title columns into the 'Title' column
Art_rating_num['Title'] = Art_rating_num[['Title_ID1', 'Title_ID2', 'Title_ID3', 'Title_ID4', 'Title_ID5']].stack().reset_index(drop=True)
Art_rating_num['rating_Count'] = Art_rating_num[['num_of_rating1', 'num_of_rating2', 'num_of_rating3', 'num_of_rating4', 'num_of_rating5']].stack().reset_index(drop=True)

# Group the titles and calculate the count
title_counts = Art_rating_num['Title'].value_counts().reset_index()
title_counts.columns = ['Title', 'rating_Count']

title_counts.head(10)

,Title,rating_Count
0,Untitled,247
1,Composition,79
2,Serpent de mer,51
3,The Dancing Lesson,51
4,Dutch Ships Ramming Spanish Galleys off the Fl...,51
5,Jouissance de l&#39;espace,51
6,Lamentation Over the Dead Christ,51
7,Equestrian statue of Gattamelata at Padua,49
8,Portrait of the Artist,49
9,Percolator,49


In [470]:
final_df =  New_Cont.merge(title_counts, on ='Title')

In [471]:
final_df.head(10)


,ID,Title,Face/body,Tags,clusters,rating_Count
0,57727de7edc2cb3880e91f26,The Defence of Sarago&#231;a,face,"Post Renaissance Art, Romanticism, David Wilkie",27,8
1,5772843bedc2cb3880fd334d,One Ton Prop (House of Cards),none,"Contemporary Art, Minimalism, Richard Serra",137,9
2,5772849eedc2cb3880fe72ab,Untitled,none,"Modern Art, Color Field Painting, Ron Gorchov",76,247
3,577288deedc2cb38800bdf0f,Untitled,none,"Modern Art, Art Informel, Karl Otto Gotz",54,247
4,577288f1edc2cb38800bff1f,Untitled,none,"Contemporary Art, Minimalism, Nasreen Mohamedi",137,247
5,57728742edc2cb388006c728,Untitled,none,"Modern Art, Art Informel, Antonio Areal",14,247
6,577287acedc2cb388007f855,Untitled,face,"Modern Art, Pop Art, Samuel Buri",59,247
7,57728437edc2cb3880fd2cbf,Untitled,face,"Modern Art, Art Informel, Helmut Sturm",14,247
8,577283cdedc2cb3880fbddf4,Untitled,none,"Modern Art, Color Field Painting, Edward Corbett",76,247
9,5772845cedc2cb3880fdbb71,Untitled,none,"Modern Art, Art Informel, Jean Miotte",4,247


In [472]:
final_df.shape

(1000, 6)

In [473]:
#remove titles with only one rating
final_df = final_df[final_df['rating_Count']>=5].reset_index(drop=True)
final_df.head()

,ID,Title,Face/body,Tags,clusters,rating_Count
0,57727de7edc2cb3880e91f26,The Defence of Sarago&#231;a,face,"Post Renaissance Art, Romanticism, David Wilkie",27,8
1,5772843bedc2cb3880fd334d,One Ton Prop (House of Cards),none,"Contemporary Art, Minimalism, Richard Serra",137,9
2,5772849eedc2cb3880fe72ab,Untitled,none,"Modern Art, Color Field Painting, Ron Gorchov",76,247
3,577288deedc2cb38800bdf0f,Untitled,none,"Modern Art, Art Informel, Karl Otto Gotz",54,247
4,577288f1edc2cb38800bff1f,Untitled,none,"Contemporary Art, Minimalism, Nasreen Mohamedi",137,247


In [474]:
final_df.shape

(995, 6)

In [475]:
final_df.drop_duplicates(['Title'])

,ID,Title,Face/body,Tags,clusters,rating_Count
0,57727de7edc2cb3880e91f26,The Defence of Sarago&#231;a,face,"Post Renaissance Art, Romanticism, David Wilkie",27,8
1,5772843bedc2cb3880fd334d,One Ton Prop (House of Cards),none,"Contemporary Art, Minimalism, Richard Serra",137,9
2,5772849eedc2cb3880fe72ab,Untitled,none,"Modern Art, Color Field Painting, Ron Gorchov",76,247
143,577284a7edc2cb3880fe813a,Dances at the Spring,none,"Modern Art, Cubism, Francis Picabia",115,10
144,577287d9edc2cb38800895a7,Self Portrait with Cloud &amp; Cigarette,face,"Modern Art, Neo-Expressionism, Joan Brown",140,10
...,...,...,...,...,...,...
990,57728a36edc2cb3880105816,"John Philip Kemble (1757‚Äì1823), as Richard i...",face,"Post Renaissance Art, Neoclassicism, William H...",39,9
991,57727594edc2cb3880ceb3d5,La Goulue Arriving at the Moulin Rouge with Tw...,face,"Modern Art, Post-Impressionism, Henri de Toulo...",32,8
992,57726ea6edc2cb3880b7e8bb,The Gleaners,face,"Post Renaissance Art, Realism, Jean-Francois M...",119,25
993,577286caedc2cb388005687d,Samson tearing the lion&#39;s mouth,face,"Post Renaissance Art, Rococo, Johann Georg Pinzel",192,8


In [476]:
final_df.shape

(995, 6)

In [477]:
only_Id = final_df[['ID']]
only_Id.head()

,ID
0,57727de7edc2cb3880e91f26
1,5772843bedc2cb3880fd334d
2,5772849eedc2cb3880fe72ab
3,577288deedc2cb38800bdf0f
4,577288f1edc2cb38800bff1f


In [478]:
only_Id.shape

(995, 1)

In [479]:
merged_df_ID = merged_df[['ID1','ID2','ID3','ID4','ID5']]

In [480]:
merged_df_ID.head()

,ID1,ID2,ID3,ID4,ID5
0,577284aeedc2cb3880fe8ca7,57728500edc2cb3880ffb2bc,57728030edc2cb3880f07e00,57726f0dedc2cb3880b94625,57728703edc2cb3880061039
1,577284aeedc2cb3880fe8ca7,57728500edc2cb3880ffb2bc,57728030edc2cb3880f07e00,57726f0dedc2cb3880b94625,57728703edc2cb3880061039
2,577284aeedc2cb3880fe8ca7,57728500edc2cb3880ffb2bc,57728030edc2cb3880f07e00,57726f0dedc2cb3880b94625,57728703edc2cb3880061039
3,577284aeedc2cb3880fe8ca7,57728500edc2cb3880ffb2bc,57728030edc2cb3880f07e00,57726f0dedc2cb3880b94625,57728703edc2cb3880061039
4,577284aeedc2cb3880fe8ca7,57728500edc2cb3880ffb2bc,57728030edc2cb3880f07e00,57726f0dedc2cb3880b94625,57728703edc2cb3880061039


In [481]:
merged_df.shape

(9279, 26)

In [482]:
# Get the unique IDs from df2
unique_ids = only_Id['ID'].unique()

# Filter df1 based on the presence of IDs in df2
filtered_df1 = merged_df_ID[merged_df_ID[['ID1','ID2','ID3','ID4','ID5']].isin(unique_ids)]
print(filtered_df1)

                           ID1                       ID2  \
0     577284aeedc2cb3880fe8ca7  57728500edc2cb3880ffb2bc   
1     577284aeedc2cb3880fe8ca7  57728500edc2cb3880ffb2bc   
2     577284aeedc2cb3880fe8ca7  57728500edc2cb3880ffb2bc   
3     577284aeedc2cb3880fe8ca7  57728500edc2cb3880ffb2bc   
4     577284aeedc2cb3880fe8ca7  57728500edc2cb3880ffb2bc   
...                        ...                       ...   
9274  5772853eedc2cb38800069d2                       NaN   
9275  5772853eedc2cb38800069d2                       NaN   
9276  5772853eedc2cb38800069d2                       NaN   
9277  5772853eedc2cb38800069d2                       NaN   
9278  5772853eedc2cb38800069d2                       NaN   

                           ID3                       ID4  \
0     57728030edc2cb3880f07e00  57726f0dedc2cb3880b94625   
1     57728030edc2cb3880f07e00  57726f0dedc2cb3880b94625   
2     57728030edc2cb3880f07e00  57726f0dedc2cb3880b94625   
3     57728030edc2cb3880f07e00  57726f0

In [483]:
filtered_df1 = filtered_df1.dropna()
print(filtered_df1)

                           ID1                       ID2  \
0     577284aeedc2cb3880fe8ca7  57728500edc2cb3880ffb2bc   
1     577284aeedc2cb3880fe8ca7  57728500edc2cb3880ffb2bc   
2     577284aeedc2cb3880fe8ca7  57728500edc2cb3880ffb2bc   
3     577284aeedc2cb3880fe8ca7  57728500edc2cb3880ffb2bc   
4     577284aeedc2cb3880fe8ca7  57728500edc2cb3880ffb2bc   
...                        ...                       ...   
9212  577284ffedc2cb3880ffb20c  57728392edc2cb3880fb3068   
9213  577284ffedc2cb3880ffb20c  57728392edc2cb3880fb3068   
9214  577284ffedc2cb3880ffb20c  57728392edc2cb3880fb3068   
9215  577284ffedc2cb3880ffb20c  57728392edc2cb3880fb3068   
9216  577284ffedc2cb3880ffb20c  57728392edc2cb3880fb3068   

                           ID3                       ID4  \
0     57728030edc2cb3880f07e00  57726f0dedc2cb3880b94625   
1     57728030edc2cb3880f07e00  57726f0dedc2cb3880b94625   
2     57728030edc2cb3880f07e00  57726f0dedc2cb3880b94625   
3     57728030edc2cb3880f07e00  57726f0

In [484]:
filtered_df1.head()

,ID1,ID2,ID3,ID4,ID5
0,577284aeedc2cb3880fe8ca7,57728500edc2cb3880ffb2bc,57728030edc2cb3880f07e00,57726f0dedc2cb3880b94625,57728703edc2cb3880061039
1,577284aeedc2cb3880fe8ca7,57728500edc2cb3880ffb2bc,57728030edc2cb3880f07e00,57726f0dedc2cb3880b94625,57728703edc2cb3880061039
2,577284aeedc2cb3880fe8ca7,57728500edc2cb3880ffb2bc,57728030edc2cb3880f07e00,57726f0dedc2cb3880b94625,57728703edc2cb3880061039
3,577284aeedc2cb3880fe8ca7,57728500edc2cb3880ffb2bc,57728030edc2cb3880f07e00,57726f0dedc2cb3880b94625,57728703edc2cb3880061039
4,577284aeedc2cb3880fe8ca7,57728500edc2cb3880ffb2bc,57728030edc2cb3880f07e00,57726f0dedc2cb3880b94625,57728703edc2cb3880061039


In [485]:
filtered_df1.shape

(1870, 5)

In [486]:
merged_df.head(2)

,Worker ID,ID1,Title_ID1,Tags_ID1,Emotions1,rating1,ID2,Title_ID2,Tags_ID2,Emotions2,...,ID4,Title_ID4,Tags_ID4,Emotions4,rating4,ID5,Title_ID5,Tags_ID5,Emotions5,rating5
0,worker_1,577284aeedc2cb3880fe8ca7,The Fight between the Lapiths and the Centaurs,"Renaissance Art, High Renaissance, Piero di Co...",anticipation,0,57728500edc2cb3880ffb2bc,Rhythmic Composition,"Modern Art, Art Informel, Ernst Wilhelm Nay",disgust,...,57726f0dedc2cb3880b94625,The Love Letter,"Post Renaissance Art, Rococo, Jean-Honore Frag...",happiness\r\nlove,3,57728703edc2cb3880061039,Sacred Flame,"Modern Art, Lyrical Abstraction, Kazuo Shiraga",trust,3
1,worker_3,577284aeedc2cb3880fe8ca7,The Fight between the Lapiths and the Centaurs,"Renaissance Art, High Renaissance, Piero di Co...",fear\r\ndisagreeableness,0,57728500edc2cb3880ffb2bc,Rhythmic Composition,"Modern Art, Art Informel, Ernst Wilhelm Nay",disgust,...,57726f0dedc2cb3880b94625,The Love Letter,"Post Renaissance Art, Rococo, Jean-Honore Frag...",happiness\r\nlove,1,57728703edc2cb3880061039,Sacred Flame,"Modern Art, Lyrical Abstraction, Kazuo Shiraga",disgust,0


In [487]:
merged_df.shape

(9279, 26)

In [488]:
final_data =  merged_df.merge(filtered_df1, on =['ID1','ID2','ID3','ID4','ID5'])

In [489]:
final_data.head()


,Worker ID,ID1,Title_ID1,Tags_ID1,Emotions1,rating1,ID2,Title_ID2,Tags_ID2,Emotions2,...,ID4,Title_ID4,Tags_ID4,Emotions4,rating4,ID5,Title_ID5,Tags_ID5,Emotions5,rating5
0,worker_1,577284aeedc2cb3880fe8ca7,The Fight between the Lapiths and the Centaurs,"Renaissance Art, High Renaissance, Piero di Co...",anticipation,0,57728500edc2cb3880ffb2bc,Rhythmic Composition,"Modern Art, Art Informel, Ernst Wilhelm Nay",disgust,...,57726f0dedc2cb3880b94625,The Love Letter,"Post Renaissance Art, Rococo, Jean-Honore Frag...",happiness\r\nlove,3,57728703edc2cb3880061039,Sacred Flame,"Modern Art, Lyrical Abstraction, Kazuo Shiraga",trust,3
1,worker_1,577284aeedc2cb3880fe8ca7,The Fight between the Lapiths and the Centaurs,"Renaissance Art, High Renaissance, Piero di Co...",anticipation,0,57728500edc2cb3880ffb2bc,Rhythmic Composition,"Modern Art, Art Informel, Ernst Wilhelm Nay",disgust,...,57726f0dedc2cb3880b94625,The Love Letter,"Post Renaissance Art, Rococo, Jean-Honore Frag...",happiness\r\nlove,3,57728703edc2cb3880061039,Sacred Flame,"Modern Art, Lyrical Abstraction, Kazuo Shiraga",trust,3
2,worker_1,577284aeedc2cb3880fe8ca7,The Fight between the Lapiths and the Centaurs,"Renaissance Art, High Renaissance, Piero di Co...",anticipation,0,57728500edc2cb3880ffb2bc,Rhythmic Composition,"Modern Art, Art Informel, Ernst Wilhelm Nay",disgust,...,57726f0dedc2cb3880b94625,The Love Letter,"Post Renaissance Art, Rococo, Jean-Honore Frag...",happiness\r\nlove,3,57728703edc2cb3880061039,Sacred Flame,"Modern Art, Lyrical Abstraction, Kazuo Shiraga",trust,3
3,worker_1,577284aeedc2cb3880fe8ca7,The Fight between the Lapiths and the Centaurs,"Renaissance Art, High Renaissance, Piero di Co...",anticipation,0,57728500edc2cb3880ffb2bc,Rhythmic Composition,"Modern Art, Art Informel, Ernst Wilhelm Nay",disgust,...,57726f0dedc2cb3880b94625,The Love Letter,"Post Renaissance Art, Rococo, Jean-Honore Frag...",happiness\r\nlove,3,57728703edc2cb3880061039,Sacred Flame,"Modern Art, Lyrical Abstraction, Kazuo Shiraga",trust,3
4,worker_1,577284aeedc2cb3880fe8ca7,The Fight between the Lapiths and the Centaurs,"Renaissance Art, High Renaissance, Piero di Co...",anticipation,0,57728500edc2cb3880ffb2bc,Rhythmic Composition,"Modern Art, Art Informel, Ernst Wilhelm Nay",disgust,...,57726f0dedc2cb3880b94625,The Love Letter,"Post Renaissance Art, Rococo, Jean-Honore Frag...",happiness\r\nlove,3,57728703edc2cb3880061039,Sacred Flame,"Modern Art, Lyrical Abstraction, Kazuo Shiraga",trust,3


In [490]:
final_data.drop_duplicates(['Worker ID'])
#This table shows a list of all the titles with more than 5 total ratings and the ratings given to them by each worker who has rated more than 50 titles in

,Worker ID,ID1,Title_ID1,Tags_ID1,Emotions1,rating1,ID2,Title_ID2,Tags_ID2,Emotions2,...,ID4,Title_ID4,Tags_ID4,Emotions4,rating4,ID5,Title_ID5,Tags_ID5,Emotions5,rating5
0,worker_1,577284aeedc2cb3880fe8ca7,The Fight between the Lapiths and the Centaurs,"Renaissance Art, High Renaissance, Piero di Co...",anticipation,0,57728500edc2cb3880ffb2bc,Rhythmic Composition,"Modern Art, Art Informel, Ernst Wilhelm Nay",disgust,...,57726f0dedc2cb3880b94625,The Love Letter,"Post Renaissance Art, Rococo, Jean-Honore Frag...",happiness\r\nlove,3,57728703edc2cb3880061039,Sacred Flame,"Modern Art, Lyrical Abstraction, Kazuo Shiraga",trust,3
9,worker_3,577284aeedc2cb3880fe8ca7,The Fight between the Lapiths and the Centaurs,"Renaissance Art, High Renaissance, Piero di Co...",fear\r\ndisagreeableness,0,57728500edc2cb3880ffb2bc,Rhythmic Composition,"Modern Art, Art Informel, Ernst Wilhelm Nay",disgust,...,57726f0dedc2cb3880b94625,The Love Letter,"Post Renaissance Art, Rococo, Jean-Honore Frag...",happiness\r\nlove,1,57728703edc2cb3880061039,Sacred Flame,"Modern Art, Lyrical Abstraction, Kazuo Shiraga",disgust,0
18,worker_4,577284aeedc2cb3880fe8ca7,The Fight between the Lapiths and the Centaurs,"Renaissance Art, High Renaissance, Piero di Co...",anger,3,57728500edc2cb3880ffb2bc,Rhythmic Composition,"Modern Art, Art Informel, Ernst Wilhelm Nay",surprise,...,57726f0dedc2cb3880b94625,The Love Letter,"Post Renaissance Art, Rococo, Jean-Honore Frag...",happiness\r\noptimism,3,57728703edc2cb3880061039,Sacred Flame,"Modern Art, Lyrical Abstraction, Kazuo Shiraga",surprise,2
27,worker_5,577284aeedc2cb3880fe8ca7,The Fight between the Lapiths and the Centaurs,"Renaissance Art, High Renaissance, Piero di Co...",anger\r\nsadness,3,57728500edc2cb3880ffb2bc,Rhythmic Composition,"Modern Art, Art Informel, Ernst Wilhelm Nay",fear,...,57726f0dedc2cb3880b94625,The Love Letter,"Post Renaissance Art, Rococo, Jean-Honore Frag...",happiness\r\nlove,3,57728703edc2cb3880061039,Sacred Flame,"Modern Art, Lyrical Abstraction, Kazuo Shiraga",love\r\nanger,3
36,worker_6,577284aeedc2cb3880fe8ca7,The Fight between the Lapiths and the Centaurs,"Renaissance Art, High Renaissance, Piero di Co...",trust,1,57728500edc2cb3880ffb2bc,Rhythmic Composition,"Modern Art, Art Informel, Ernst Wilhelm Nay",disgust,...,57726f0dedc2cb3880b94625,The Love Letter,"Post Renaissance Art, Rococo, Jean-Honore Frag...",trust,1,57728703edc2cb3880061039,Sacred Flame,"Modern Art, Lyrical Abstraction, Kazuo Shiraga",disgust,-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5043,worker_136,57726e1bedc2cb3880b5f917,The Dancing Lesson,"Post Renaissance Art, Rococo, Pietro Longhi",happiness,2,5772847bedc2cb3880fdede7,Dutch Ships Ramming Spanish Galleys off the Fl...,"Post Renaissance Art, Baroque, Hendrick Cornel...",optimism,...,57727c8fedc2cb3880e4c8b0,Lamentation Over the Dead Christ,"Renaissance Art, Early Renaissance, Carlo Criv...",humility,3,5772845bedc2cb3880fdb9df,Serpent de mer,"Modern Art, Art Informel, Rene Duvillier",anticipation,0
6063,worker_131,57726e1bedc2cb3880b5f917,The Dancing Lesson,"Post Renaissance Art, Rococo, Pietro Longhi",happiness,1,5772847bedc2cb3880fdede7,Dutch Ships Ramming Spanish Galleys off the Fl...,"Post Renaissance Art, Baroque, Hendrick Cornel...",disagreeableness,...,57727c8fedc2cb3880e4c8b0,Lamentation Over the Dead Christ,"Renaissance Art, Early Renaissance, Carlo Criv...",sadness,-1,5772845bedc2cb3880fdb9df,Serpent de mer,"Modern Art, Art Informel, Rene Duvillier",disgust,-2
6764,worker_127,57727c8eedc2cb3880e4c790,Annunciation with Saint Emidius,"Renaissance Art, Early Renaissance, Carlo Criv...",happiness\r\nlove,0,57728649edc2cb388003a6e7,Chateaubriand Meditating on the Ruins of Rome,"Post Renaissance Art, Romanticism, Anne-Louis ...",sadness,...,577286cdedc2cb3880056d83,Opus 4,"Modern Art, Abstract Art, Victor Servranckx",optimism,2,5772839fedc2cb3880fb43f5,Untitled,"Contemporary Art, Minimalism, Keith Sonnier",happiness\r\n

In [491]:
final_data.shape

(39216, 26)

In [492]:
Title_1 = final_data[['Worker ID','ID1','Title_ID1','Tags_ID1','rating1']]
Title_1.drop_duplicates(['Worker ID'])
Title_1=Title_1.rename(columns={'ID1': 'ID','Title_ID1': 'Title','Tags_ID1': 'Tags','rating1': 'rating'})
Title_1.head()

,Worker ID,ID,Title,Tags,rating
0,worker_1,577284aeedc2cb3880fe8ca7,The Fight between the Lapiths and the Centaurs,"Renaissance Art, High Renaissance, Piero di Co...",0
1,worker_1,577284aeedc2cb3880fe8ca7,The Fight between the Lapiths and the Centaurs,"Renaissance Art, High Renaissance, Piero di Co...",0
2,worker_1,577284aeedc2cb3880fe8ca7,The Fight between the Lapiths and the Centaurs,"Renaissance Art, High Renaissance, Piero di Co...",0
3,worker_1,577284aeedc2cb3880fe8ca7,The Fight between the Lapiths and the Centaurs,"Renaissance Art, High Renaissance, Piero di Co...",0
4,worker_1,577284aeedc2cb3880fe8ca7,The Fight between the Lapiths and the Centaurs,"Renaissance Art, High Renaissance, Piero di Co...",0


In [493]:
Title_2 = final_data[['Worker ID','ID2','Title_ID2','Tags_ID2','rating2']]
Title_2.drop_duplicates(['Worker ID'])
Title_2=Title_2.rename(columns={'ID2': 'ID','Title_ID2': 'Title','Tags_ID2': 'Tags','rating2': 'rating'})
Title_2.head()

,Worker ID,ID,Title,Tags,rating
0,worker_1,57728500edc2cb3880ffb2bc,Rhythmic Composition,"Modern Art, Art Informel, Ernst Wilhelm Nay",-2
1,worker_1,57728500edc2cb3880ffb2bc,Rhythmic Composition,"Modern Art, Art Informel, Ernst Wilhelm Nay",-2
2,worker_1,57728500edc2cb3880ffb2bc,Rhythmic Composition,"Modern Art, Art Informel, Ernst Wilhelm Nay",-2
3,worker_1,57728500edc2cb3880ffb2bc,Rhythmic Composition,"Modern Art, Art Informel, Ernst Wilhelm Nay",-2
4,worker_1,57728500edc2cb3880ffb2bc,Rhythmic Composition,"Modern Art, Art Informel, Ernst Wilhelm Nay",-2


In [494]:
Title_3 = final_data[['Worker ID','ID3','Title_ID3','Tags_ID3','rating3']]
Title_3.drop_duplicates(['Worker ID'])
Title_3=Title_3.rename(columns={'ID3': 'ID','Title_ID3': 'Title','Tags_ID3': 'Tags','rating3': 'rating'})
Title_3.head()

,Worker ID,ID,Title,Tags,rating
0,worker_1,57728030edc2cb3880f07e00,A Wall for Apricots,"Contemporary Art, Minimalism, Anne Truitt",0
1,worker_1,57728030edc2cb3880f07e00,A Wall for Apricots,"Contemporary Art, Minimalism, Anne Truitt",0
2,worker_1,57728030edc2cb3880f07e00,A Wall for Apricots,"Contemporary Art, Minimalism, Anne Truitt",0
3,worker_1,57728030edc2cb3880f07e00,A Wall for Apricots,"Contemporary Art, Minimalism, Anne Truitt",0
4,worker_1,57728030edc2cb3880f07e00,A Wall for Apricots,"Contemporary Art, Minimalism, Anne Truitt",0


In [495]:
Title_4 = final_data[['Worker ID','ID4','Title_ID4','Tags_ID4','rating4']]
Title_4.drop_duplicates(['Worker ID'])
Title_4=Title_4.rename(columns={'ID4': 'ID','Title_ID4': 'Title','Tags_ID4': 'Tags','rating4': 'rating'})
Title_4.head()

,Worker ID,ID,Title,Tags,rating
0,worker_1,57726f0dedc2cb3880b94625,The Love Letter,"Post Renaissance Art, Rococo, Jean-Honore Frag...",3
1,worker_1,57726f0dedc2cb3880b94625,The Love Letter,"Post Renaissance Art, Rococo, Jean-Honore Frag...",3
2,worker_1,57726f0dedc2cb3880b94625,The Love Letter,"Post Renaissance Art, Rococo, Jean-Honore Frag...",3
3,worker_1,57726f0dedc2cb3880b94625,The Love Letter,"Post Renaissance Art, Rococo, Jean-Honore Frag...",3
4,worker_1,57726f0dedc2cb3880b94625,The Love Letter,"Post Renaissance Art, Rococo, Jean-Honore Frag...",3


In [496]:
Title_5 = final_data[['Worker ID','ID5','Title_ID5','Tags_ID5','rating5']]
Title_5.drop_duplicates(['Worker ID'])
Title_5=Title_5.rename(columns={'ID5': 'ID','Title_ID5': 'Title','Tags_ID5': 'Tags','rating5': 'rating'})
Title_5.head()

,Worker ID,ID,Title,Tags,rating
0,worker_1,57728703edc2cb3880061039,Sacred Flame,"Modern Art, Lyrical Abstraction, Kazuo Shiraga",3
1,worker_1,57728703edc2cb3880061039,Sacred Flame,"Modern Art, Lyrical Abstraction, Kazuo Shiraga",3
2,worker_1,57728703edc2cb3880061039,Sacred Flame,"Modern Art, Lyrical Abstraction, Kazuo Shiraga",3
3,worker_1,57728703edc2cb3880061039,Sacred Flame,"Modern Art, Lyrical Abstraction, Kazuo Shiraga",3
4,worker_1,57728703edc2cb3880061039,Sacred Flame,"Modern Art, Lyrical Abstraction, Kazuo Shiraga",3


In [497]:

Title_1 = Title_1.set_index('ID')
Title_2 = Title_2.set_index('ID')
Title_3 = Title_3.set_index('ID')
Title_4 = Title_4.set_index('ID')
Title_5 = Title_5.set_index('ID')

# Concatenate the dataframes vertically (stacked)
merged_df = pd.concat([Title_1, Title_2, Title_3, Title_4, Title_5])

# Reset the index to convert 'ID' back to a regular column
merged_df = merged_df.reset_index()
merged_df.head()

,ID,Worker ID,Title,Tags,rating
0,577284aeedc2cb3880fe8ca7,worker_1,The Fight between the Lapiths and the Centaurs,"Renaissance Art, High Renaissance, Piero di Co...",0
1,577284aeedc2cb3880fe8ca7,worker_1,The Fight between the Lapiths and the Centaurs,"Renaissance Art, High Renaissance, Piero di Co...",0
2,577284aeedc2cb3880fe8ca7,worker_1,The Fight between the Lapiths and the Centaurs,"Renaissance Art, High Renaissance, Piero di Co...",0
3,577284aeedc2cb3880fe8ca7,worker_1,The Fight between the Lapiths and the Centaurs,"Renaissance Art, High Renaissance, Piero di Co...",0
4,577284aeedc2cb3880fe8ca7,worker_1,The Fight between the Lapiths and the Centaurs,"Renaissance Art, High Renaissance, Piero di Co...",0


In [498]:
merged_df.shape

(196080, 5)

### PivotTable to group Users

In [528]:
Pivot = merged_df.pivot_table(columns = {'Worker ID'}, index = ['Title'], values = ['rating'])

In [529]:
Pivot

rating                       \
Worker ID                                   worker_1 worker_10 worker_101   
Title                                                                       
#17DR                                            NaN       NaN        NaN   
&quot;Cool&quot; Series No. 26 (First Love)      NaN       2.0        3.0   
18 f&#233;vrier                                  1.0       NaN        NaN   
1957-D No. 1                                     NaN       NaN        NaN   
A Fortune Teller at Venice                       NaN       1.0        NaN   
...                                              ...       ...        ...   
Yellow Triptych                                  NaN       NaN        NaN   
Yellow man with Trolley                          1.0       1.0        NaN   
Yellow, Cherry,&#160;Orange                      NaN       NaN        NaN   
Zone                                             NaN       NaN        NaN   
composition                                      NaN       NaN        NaN   

                                                                              \
Worker ID                                   worker_103 worker_104 worker_107   
Title                                                                          
#17DR                                              NaN        NaN       -2.0   
&quot;Cool&quot; Series No. 26 (First Love)        NaN        NaN        NaN   
18 f&#233;vrier                                    NaN        NaN        NaN   
1957-D No. 1                                       NaN        NaN        NaN   
A Fortune Teller at Venice                         NaN        NaN        NaN   
...                                                ...        ...        ...   
Yellow Triptych                                    NaN        NaN        NaN   
Yellow man with Trolley                            NaN        3.0        NaN   
Yellow, Cherry,&#160;Orange                        NaN        NaN        NaN   
Zone                                               NaN        NaN        NaN   
composition                                        NaN        NaN        NaN   

                                                                             \
Worker ID                                   worker_108 worker_11 worker_111   
Title                                                                         
#17DR                                              NaN       NaN        NaN   
&quot;Cool&quot; Series No. 26 (First Love)        NaN       NaN        NaN   
18 f&#233;vrier                                    NaN       NaN        NaN   
1957-D No. 1                                       NaN       NaN        NaN   
A Fortune Teller at Venice                         NaN       NaN        NaN   
...                                                ...       ...        ...   
Yellow Triptych                                    2.0       NaN        NaN   
Yellow man with Trolley                            NaN       NaN        0.0   
Yellow, Cherry,&#160;Orange                        2.0       NaN        0.0   
Zone                                               NaN       NaN        NaN   
composition                                        NaN       NaN        NaN   

                                                        ...           \
Worker ID                                   worker_114  ... worker_8   
Title                                                   ...            
#17DR                                              NaN  ...      NaN   
&quot;Cool&quot; Series No. 26 (First Love)        NaN  ...      NaN   
18 f&#233;vrier                                    NaN  ...      1.0   
1957-D No. 1                                       NaN  ...      1.0   
A Fortune Teller at Venice                         NaN  ...      1.0   
...                                                ...  ...      ...   
Yellow Triptych                                    NaN  ...      1.0   
Yellow man with Tro

In [530]:
Pivot.shape

(721, 75)

In [531]:
Pivot.fillna(0, inplace=True)
Pivot

rating                       \
Worker ID                                   worker_1 worker_10 worker_101   
Title                                                                       
#17DR                                            0.0       0.0        0.0   
&quot;Cool&quot; Series No. 26 (First Love)      0.0       2.0        3.0   
18 f&#233;vrier                                  1.0       0.0        0.0   
1957-D No. 1                                     0.0       0.0        0.0   
A Fortune Teller at Venice                       0.0       1.0        0.0   
...                                              ...       ...        ...   
Yellow Triptych                                  0.0       0.0        0.0   
Yellow man with Trolley                          1.0       1.0        0.0   
Yellow, Cherry,&#160;Orange                      0.0       0.0        0.0   
Zone                                             0.0       0.0        0.0   
composition                                      0.0       0.0        0.0   

                                                                              \
Worker ID                                   worker_103 worker_104 worker_107   
Title                                                                          
#17DR                                              0.0        0.0       -2.0   
&quot;Cool&quot; Series No. 26 (First Love)        0.0        0.0        0.0   
18 f&#233;vrier                                    0.0        0.0        0.0   
1957-D No. 1                                       0.0        0.0        0.0   
A Fortune Teller at Venice                         0.0        0.0        0.0   
...                                                ...        ...        ...   
Yellow Triptych                                    0.0        0.0        0.0   
Yellow man with Trolley                            0.0        3.0        0.0   
Yellow, Cherry,&#160;Orange                        0.0        0.0        0.0   
Zone                                               0.0        0.0        0.0   
composition                                        0.0        0.0        0.0   

                                                                             \
Worker ID                                   worker_108 worker_11 worker_111   
Title                                                                         
#17DR                                              0.0       0.0        0.0   
&quot;Cool&quot; Series No. 26 (First Love)        0.0       0.0        0.0   
18 f&#233;vrier                                    0.0       0.0        0.0   
1957-D No. 1                                       0.0       0.0        0.0   
A Fortune Teller at Venice                         0.0       0.0        0.0   
...                                                ...       ...        ...   
Yellow Triptych                                    2.0       0.0        0.0   
Yellow man with Trolley                            0.0       0.0        0.0   
Yellow, Cherry,&#160;Orange                        2.0       0.0        0.0   
Zone                                               0.0       0.0        0.0   
composition                                        0.0       0.0        0.0   

                                                        ...           \
Worker ID                                   worker_114  ... worker_8   
Title                                                   ...            
#17DR                                              0.0  ...      0.0   
&quot;Cool&quot; Series No. 26 (First Love)        0.0  ...      0.0   
18 f&#233;vrier                                    0.0  ...      1.0   
1957-D No. 1                                       0.0  ...      1.0   
A Fortune Teller at Venice                         0.0  ...      1.0   
...                                                ...  ...      ...   
Yellow Triptych                                    0.0  ...      1.0   
Yellow man with Tro

In [532]:

# Create a new DataFrame to store the modified pivot table
new_pivot = pd.DataFrame(index=Pivot.index)

# Add a row with index numbers
new_pivot['Index'] = range(len(Pivot))

# Concatenate the index row to the pivot table
final_pivot = pd.concat([new_pivot, Pivot], axis=1)

# Save all the indexes in an array
indexes = final_pivot['Index'].values

final_pivot.head()

,Index,"(rating, worker_1)","(rating, worker_10)","(rating, worker_101)","(rating, worker_103)","(rating, worker_104)","(rating, worker_107)","(rating, worker_108)","(rating, worker_11)","(rating, worker_111)",...,"(rating, worker_8)","(rating, worker_81)","(rating, worker_83)","(rating, worker_84)","(rating, worker_87)","(rating, worker_9)","(rating, worker_90)","(rating, worker_91)","(rating, worker_92)","(rating, worker_99)"
Title,,,,,,,,,,,,,,,,,,,,,
#17DR,0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
&quot;Cool&quot; Series No. 26 (First Love),1,0.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-2.0,0.0,0.0,0.0
18 f&#233;vrier,2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1957-D No. 1,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Fortune Teller at Venice,4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


CSR Matrix

In [533]:
from scipy.sparse import csr_matrix

In [534]:
Pivot.sparse = csr_matrix(Pivot)
Pivot.sparse

<721x75 sparse matrix of type '<class 'numpy.float64'>'
	with 7141 stored elements in Compressed Sparse Row format>

In [535]:
print(Pivot.sparse)

  (0, 5)	-2.0
  (0, 11)	2.0
  (0, 15)	1.0
  (0, 44)	2.0
  (0, 51)	2.0
  (0, 53)	2.0
  (0, 72)	2.0
  (1, 1)	2.0
  (1, 2)	3.0
  (1, 19)	-3.0
  (1, 35)	1.0
  (1, 37)	-1.0
  (1, 71)	-2.0
  (2, 0)	1.0
  (2, 24)	-1.0
  (2, 39)	1.0
  (2, 65)	1.0
  (2, 69)	1.0
  (3, 20)	1.0
  (3, 36)	1.0
  (3, 39)	2.0
  (3, 41)	-2.0
  (3, 43)	-2.0
  (3, 47)	-3.0
  (3, 65)	1.0
  :	:
  (717, 56)	1.5
  (717, 57)	1.0
  (717, 65)	1.0
  (717, 67)	2.0
  (717, 68)	1.0
  (717, 69)	2.0
  (717, 71)	-2.0
  (718, 6)	2.0
  (718, 37)	-1.0
  (718, 62)	-3.0
  (718, 63)	-1.0
  (718, 65)	1.0
  (718, 67)	2.0
  (719, 41)	-2.0
  (719, 43)	-1.0
  (719, 62)	-3.0
  (719, 65)	1.0
  (719, 67)	2.0
  (720, 9)	3.0
  (720, 18)	2.0
  (720, 24)	-1.0
  (720, 26)	-2.0
  (720, 40)	-1.0
  (720, 57)	-3.0
  (720, 70)	-3.0


In [536]:
from sklearn.neighbors import NearestNeighbors

In [537]:
model = NearestNeighbors(metric = 'cosine', algorithm='brute')
model.fit(Pivot.sparse)

NearestNeighbors(algorithm='brute', metric='cosine')

In [538]:
distance, suggestion = model.kneighbors(Pivot.iloc[12,:].values.reshape(1,-1), n_neighbors=6)

In [539]:
distance

array([[0.        , 0.26039974, 0.38196856, 0.45529522, 0.46068063,
        0.4954954 ]])

In [540]:
suggestion

array([[ 12, 455, 439, 130, 253, 147]])

In [541]:
for i in range(len (suggestion)):
  print(Pivot.index[suggestion[i]])

Index(['A Youth with a Jug', 'Rape of the Daughters of Leucippus',
       'Portrait of a Lady, thought to be Anne Boleyn', 'Crimson Glory',
       'Kids on Bikes', 'Don Quixote and Sancho Panza in the Mountains'],
      dtype='object', name='Title')


In [542]:
Pivot.index

Index(['#17DR', '&quot;Cool&quot; Series No. 26 (First Love)',
       '18 f&#233;vrier', '1957-D No. 1', 'A Fortune Teller at Venice',
       'A Quiet Moment', 'A Saddled Race Horse Tied to a Fence',
       'A Scene from &#39;As You Like It&#39; by William Shakespeare',
       'A Studio in the Batignolles (Homage to Manet)',
       'A View of the Tiber with Rome in the Distance',
       ...
       'Woman in an Interior with a Mirror', 'Work', 'Writing',
       'Yawning Apprentice', 'Yellow Sky', 'Yellow Triptych',
       'Yellow man with Trolley', 'Yellow, Cherry,&#160;Orange', 'Zone',
       'composition'],
      dtype='object', name='Title', length=721)

In [543]:
Titles=Pivot.index

In [544]:
Titles.shape

(721,)

In [545]:
def collab(title_index):
    distance, suggestion = model.kneighbors(Pivot.iloc[title_index, :].values.reshape(1, -1), n_neighbors=9)

    input_title = Pivot.index[title_index]

    print(f"Recommendations for index {title_index} ({input_title}):")
    for i, index in enumerate(suggestion[0]):
        if Pivot.index[index] != input_title:
            print(f"  Index: {index}, Title: {Pivot.index[index]}")

In [546]:
collab(5)

Recommendations for index 5 (A Quiet Moment):
  Index: 277, Title: Landscape with Trees, Santa Monica
  Index: 248, Title: John Philip Kemble (1757‚Äì1823), as Richard in &#39;Richard III&#39; by William Shakespeare
  Index: 534, Title: Still-life with Chessboard and Pipe
  Index: 116, Title: Color Test
  Index: 533, Title: Still life
  Index: 532, Title: Still Life with Pumpkin, Plums, Cherries, Figs and Jug
  Index: 709, Title: Woman Playing a Lute
  Index: 368, Title: OE 280 (Komposition in Rot mit Wei&#223; und Blau)


In [547]:
# Perform K-means clustering
num_clusters = 30
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
item_clusters = kmeans.fit_predict(Pivot.sparse)

# Calculate cosine similarity between items based on attributes
cosine_sim = linear_kernel(Pivot.sparse, Pivot.sparse)

# Create a dictionary to map item IDs to cluster labels
item_to_cluster = dict(zip(indexes, item_clusters))

print(item_to_cluster)

{0: 1, 1: 2, 2: 2, 3: 14, 4: 7, 5: 5, 6: 16, 7: 29, 8: 12, 9: 29, 10: 27, 11: 5, 12: 9, 13: 22, 14: 16, 15: 7, 16: 2, 17: 0, 18: 23, 19: 22, 20: 2, 21: 14, 22: 6, 23: 9, 24: 4, 25: 29, 26: 18, 27: 5, 28: 2, 29: 9, 30: 9, 31: 27, 32: 2, 33: 0, 34: 2, 35: 1, 36: 13, 37: 11, 38: 3, 39: 2, 40: 0, 41: 23, 42: 2, 43: 27, 44: 23, 45: 2, 46: 22, 47: 14, 48: 23, 49: 20, 50: 21, 51: 24, 52: 1, 53: 2, 54: 22, 55: 2, 56: 2, 57: 27, 58: 27, 59: 24, 60: 1, 61: 13, 62: 13, 63: 2, 64: 26, 65: 5, 66: 16, 67: 20, 68: 5, 69: 22, 70: 22, 71: 14, 72: 18, 73: 5, 74: 19, 75: 27, 76: 9, 77: 14, 78: 29, 79: 2, 80: 7, 81: 10, 82: 2, 83: 2, 84: 2, 85: 4, 86: 16, 87: 22, 88: 6, 89: 5, 90: 23, 91: 2, 92: 2, 93: 2, 94: 5, 95: 3, 96: 23, 97: 5, 98: 2, 99: 14, 100: 28, 101: 14, 102: 2, 103: 14, 104: 12, 105: 16, 106: 18, 107: 5, 108: 10, 109: 6, 110: 3, 111: 5, 112: 28, 113: 4, 114: 4, 115: 2, 116: 5, 117: 2, 118: 5, 119: 4, 120: 8, 121: 2, 122: 5, 123: 27, 124: 14, 125: 8, 126: 14, 127: 1, 128: 22, 129: 2, 130: 2, 1

In [548]:
index = 0  # Replace with the item ID you want to check
cluster_id = item_to_cluster.get(index)

cluster_items = []

if cluster_id is not None:
    print(f"Item {index} belongs to Cluster {cluster_id}")

    # Find indices of items in the same cluster
    cluster_items_indices = np.where(item_clusters == cluster_id)[0]

    # Exclude the input item itself from the indices
    cluster_items_indices = cluster_items_indices[cluster_items_indices != index]

    # Get the item IDs of all items in the same cluster
    items_in_cluster = [index for index in item_to_cluster.keys() if item_to_cluster[index] == cluster_id]

    # Remove the input item from the items in the cluster
    items_in_cluster.remove(index)

    # Assign the item IDs to the cluster_items list
    cluster_items.extend(items_in_cluster)
else:
    print(f"Item {index} not found in any cluster.")


print('Items in this cluster are:')
# Print the saved cluster items (only item IDs) up to the top 8
for i, item in enumerate(cluster_items[:8], start=1):
    print(f"{i}. Item ID {item}")

# Assign the top 8 cluster item IDs to a variable
cluster_item_ids = cluster_items[:8]

print(cluster_item_ids)

Item 0 belongs to Cluster 1
Items in this cluster are:
1. Item ID 35
2. Item ID 52
3. Item ID 60
4. Item ID 127
5. Item ID 137
6. Item ID 144
7. Item ID 209
8. Item ID 235
[35, 52, 60, 127, 137, 144, 209, 235]


Collaborative filtering model

In [549]:
def get_recommendations(index):
    # Get the top 8 clusters
    cluster_id = item_to_cluster.get(index)
    cluster_items = []

    if cluster_id is not None:
        # Find indices of items in the same cluster
        cluster_items_indices = np.where(item_clusters == cluster_id)[0]

        # Exclude the input item itself from the indices
        cluster_items_indices = cluster_items_indices[cluster_items_indices != index]

        # Get the item IDs of all items in the same cluster
        items_in_cluster = [item for item in item_to_cluster.keys() if item_to_cluster[item] == cluster_id]

        # Remove the input item from the items in the cluster
        items_in_cluster.remove(index)

        # Assign the item IDs to the cluster_items list
        cluster_items.extend(items_in_cluster)
    else:
        print(f"Item {index} not found in any cluster.")

    # Print the top 8 cluster items
    print('Top 8 items in this cluster are:')
    for i, item in enumerate(cluster_items[:8], start=1):
        print(f"{i}. Item ID {item}")

    # Get the top 8 recommendations using nearest neighbors
    distance, suggestion = model.kneighbors(Pivot.iloc[index, :].values.reshape(1, -1), n_neighbors=9)

    input_title = Pivot.index[index]
    print(f"Recommendations for index {index} - Title: ({input_title}):")
    for i, idx in enumerate(suggestion[0]):
        if idx != index:
            Id = main_art['ID'].iloc[index]
            title = main_art['Title'].iloc[index]
            image_url = main_art['Image URL'].iloc[index]
            style = main_art['Style'].iloc[index]
            category = main_art['Category'].iloc[index]
            artist = main_art['Artist'].iloc[index]
            print(f"  Index: {idx}")
            print(f"   ID: {Id}")
            print(f"   Title: {title}")
            print(f"   Image URL: {image_url}")
            print(f"   Style: {style}")
            print(f"   Category: {category}")
            print(f"   Artist: {artist}")
    # Calculate precision and recall
    actual_items = cluster_items[:8]
    recommended_indices = suggestion[0][1:]  # Exclude the input index

    precision = precision_score(actual_items, recommended_indices, average='micro')
    recall = recall_score(actual_items, recommended_indices, average='micro')
    f1 = f1_score(actual_items, recommended_indices, average='micro')

    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")

# Choose an input index for testing
input_index = 19

# Call the function to get recommendations and calculate precision and recall
get_recommendations(input_index)


Top 8 items in this cluster are:
1. Item ID 13
2. Item ID 46
3. Item ID 54
4. Item ID 69
5. Item ID 70
6. Item ID 87
7. Item ID 128
8. Item ID 135
Recommendations for index 19 - Title: (Admiral Sir Charles Hardy (c.1716‚Äì1780)):
  Index: 46
   ID: 577288ddedc2cb38800bde1d
   Title: Peinture-Relief
   Image URL: https://uploads4.wikiart.org/images/herbert-zangs/peinture-relief-1955.jpg
   Style: Contemporary Art
   Category: Minimalism
   Artist: Herbert Zangs
  Index: 304
   ID: 577288ddedc2cb38800bde1d
   Title: Peinture-Relief
   Image URL: https://uploads4.wikiart.org/images/herbert-zangs/peinture-relief-1955.jpg
   Style: Contemporary Art
   Category: Minimalism
   Artist: Herbert Zangs
  Index: 617
   ID: 577288ddedc2cb38800bde1d
   Title: Peinture-Relief
   Image URL: https://uploads4.wikiart.org/images/herbert-zangs/peinture-relief-1955.jpg
   Style: Contemporary Art
   Category: Minimalism
   Artist: Herbert Zangs
  Index: 69
   ID: 577288ddedc2cb38800bde1d
   Title: Peinture-

## Creating our Hybrid model

In [551]:
def hybrid_recommendation(item_ID, num_recommendations=8):
    try:
        # Find the index of the input title in Pivot
        item_index = Pivot.index.get_loc(item_ID)

        # Content-based recommendation
        cluster_id = item_to_cluster.get(item_index)
        content_recommendations = []

        if cluster_id is not None:
            cluster_items_indices = np.where(item_clusters == cluster_id)[0]
            cluster_items_indices = cluster_items_indices[cluster_items_indices != item_index]

            items_in_cluster = [item for item in item_to_cluster.keys() if item_to_cluster[item] == cluster_id]
            items_in_cluster.remove(item_index)

            sim_scores = list(enumerate(cosine_sim[item_index]))
            sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
            sim_scores = [i for i in sim_scores if i[0] in cluster_items_indices]
            sim_scores = sim_scores[:num_recommendations]

            content_recommendations = [i[0] for i in sim_scores]

        # Collaborative filtering recommendation
        distance, suggestion = model.kneighbors(Pivot.iloc[item_index, :].values.reshape(1, -1), n_neighbors=num_recommendations+1)
        collab_recommendations = suggestion[0][1:]  # Exclude the input item

        # Hybrid recommendations
        hybrid_recommendations = list(set(content_recommendations) & set(collab_recommendations))

        # Create a dictionary to store the titles and URLs
        titles_and_urls = {}

        # Print content-based recommendations
        if len(content_recommendations) > 0:
            print("Content-Based Recommendations:")
            for i, index in enumerate(content_recommendations, start=1):
                title = New_Cont['Title'].iloc[index]  # Replace 'Title' with the actual column name
                Id = main_art['ID'].iloc[index]
                image_url = main_art['Image URL'].iloc[index]
                style = main_art['Style'].iloc[index]
                category = main_art['Category'].iloc[index]
                artist = main_art['Artist'].iloc[index]
                print(f"{i + 1}. Item ID {index}")
                print(f"   ID: {Id}")
                print(f"   Title: {title}")
                print(f"   Image URL: {image_url}")
                print(f"   Style: {style}")
                print(f"   Category: {category}")
                print(f"   Artist: {artist}")
                titles_and_urls[index] = {'Title': title, 'Image URL': image_url, 'Style': style, 'Category': category, 'Artist': artist}

        # Print collaborative filtering recommendations
        if len(collab_recommendations) > 0:
            print("Collaborative Filtering Recommendations:")
            for i, index in enumerate(collab_recommendations, start=1):
                input_title = Pivot.index[index]
                Id = main_art['ID'].iloc[index]
                image_url = main_art['Image URL'].iloc[index]
                style = main_art['Style'].iloc[index]
                category = main_art['Category'].iloc[index]
                artist = main_art['Artist'].iloc[index]
                print(f"  Index: {index}")
                print(f"   ID: {Id}")
                print(f"   Title: {title}")
                print(f"   Image URL: {image_url}")
                print(f"   Style: {style}")
                print(f"   Category: {category}")
                print(f"   Artist: {artist}")
                titles_and_urls[index] = {'Title': input_title, 'Image URL': image_url, 'Style': style, 'Category': category, 'Artist': artist}

        # Check if there are fewer than 8 matches in the hybrid intersection
        if len(hybrid_recommendations) < num_recommendations:
            remaining = num_recommendations - len(hybrid_recommendations)

            # Distribute the remaining recommendations evenly between content and collaborative
            content_remaining = remaining // 2
            collab_remaining = remaining - content_remaining

            # Add recommendations from content-based filtering
            content_additional_recommendations = [item for item in content_recommendations if item not in hybrid_recommendations]
            for i in range(content_remaining):
                if i < len(content_additional_recommendations):
                    hybrid_recommendations.append(content_additional_recommendations[i])

            # Add recommendations from collaborative filtering
            collab_additional_recommendations = [item for item in collab_recommendations if item not in hybrid_recommendations]
            for i in range(collab_remaining):
                if i < len(collab_additional_recommendations):
                    hybrid_recommendations.append(collab_additional_recommendations[i])

        # Print hybrid recommendations with titles and URLs
        if len(hybrid_recommendations) > 0:
            print("Hybrid Recommendations:")
            for i, index in enumerate(hybrid_recommendations, start=1):
                item_data = titles_and_urls.get(index, {'title': 'Title not found', 'image_url': 'URL not found'})
                Id = main_art['ID'].iloc[index]
                image_url = main_art['Image URL'].iloc[index]
                style = main_art['Style'].iloc[index]
                category = main_art['Category'].iloc[index]
                artist = main_art['Artist'].iloc[index]
                #print(f"{i}. Item ID {index} - Title: {title} - Image URL: {image_url} - Style: {style}")
                print(f"  Item ID {index}")
                print(f"   ID: {Id}")
                print(f"   Title: {title}")
                print(f"   Image URL: {image_url}")
                print(f"   Style: {style}")
                print(f"   Category: {category}")
                print(f"   Artist: {artist}")

    except KeyError:
        print("Item not found in Pivot.")

# Choose an item title for testing
test_item_title = "Crimson Glory"

# Get hybrid recommendations
hybrid_recommendation(test_item_title)


Content-Based Recommendations:
2. Item ID 147
   ID: 5772842cedc2cb3880fd1bf7
   Title: Montaru 9
   Image URL: https://uploads1.wikiart.org/images/willi-baumeister/montaru-9-1953.jpg
   Style: Modern Art
   Category: Art Informel
   Artist: Willi Baumeister
3. Item ID 367
   ID: 57728108edc2cb3880f321ce
   Title: Portrait of Catherine II
   Image URL: https://uploads1.wikiart.org/images/dmitry-levitzky/portrait-of-catherine-ii.jpg
   Style: Post Renaissance Art
   Category: Neoclassicism
   Artist: Dmitry Levitzky
4. Item ID 45
   ID: 57727fb8edc2cb3880eecaef
   Title: Dunes, Oceano
   Image URL: https://uploads1.wikiart.org/images/edward-weston/dunes-oceano-1936.jpg
   Style: Modern Art
   Category: Surrealism
   Artist: Edward Weston
5. Item ID 620
   ID: 577287dbedc2cb3880089878
   Title: Light Blue Cityscape no. 4
   Image URL: https://uploads6.wikiart.org/images/martha-diamond/light-blue-cityscape-no-4-1996.jpg
   Style: Modern Art
   Category: Neo-Expressionism
   Artist: Martha